<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/5_8_robust_sharpen_bound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Classical MVP

In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
alpha0_classical_mv_unconstrained_target.py  –  α = 0 （古典的平均–分散ポートフォリオ、和の制約なし、目標リターンあり）
    • sum pi = 1 の制約を外し、目標リターン mu_tilde を達成しつつ分散を最小化するポートフォリオを計算
    • alpha_sweep_K4M4_SR.csv と互換な列構成で 1 行だけの CSV を生成
    • SR_rob には計算されたポートフォリオの Sharpe を格納
    • w*, λ, dpi* は NaN で空欄（robust 問題に固有の列のため）
"""

# ---------------------------------------------------------------------
# 0. IMPORTS & CONSTANTS
# ---------------------------------------------------------------------
import numpy as np
import pandas as pd

K, M        = 4, 4
mu_tilde    = 0.03           # 目標リターン (必要)
alpha0      = 0.0            # 古典設定

# ---------------------------------------------------------------------
# 1. SYNTHETIC DATA（元スクリプトと同一）
# ---------------------------------------------------------------------
r_base = np.array([0.02, 0.03, 0.04, 0.05])
Delta_A = np.array([[+1, 1, -1, -1],
                    [+1, 1, -1, -1],
                    [+2, -1, +2, -2],
                    [+2, -2, +2, -2]], dtype=float)
Delta_r_m     = Delta_A / 1000
sigma_base    = np.array([0.20, 0.25, 0.30, 0.35])
Delta_sigma_m = Delta_A / 100

def block_corr(rho_in: float, rho_out: float) -> np.ndarray:
    C = np.full((K, K), rho_out)
    np.fill_diagonal(C, 1.0)
    sectA, sectB = [0, 1], [2, 3]
    for i in sectA:
        for j in sectA:
            if i != j:
                C[i, j] = rho_in
    for i in sectB:
        for j in sectB:
            if i != j:
                C[i, j] = rho_in
    return C

C_base = block_corr(0.75, 0.50)
tilde_C_list = [
    block_corr(0.85, 0.40),
    block_corr(0.65, 0.40),
    block_corr(0.65, 0.60),
    block_corr(0.85, 0.60),
]

def mix_corr(C0, C1, alpha, beta=0.5, eps=1e-4):
    C = (1 - beta * alpha) * C0 + beta * alpha * C1
    eigval, eigvec = np.linalg.eigh(C)
    eigval_clip = np.clip(eigval, eps, None)
    C_spd = eigvec @ np.diag(eigval_clip) @ eigvec.T
    D = np.sqrt(np.diag(C_spd))
    return C_spd / np.outer(D, D)

def build_params(alpha: float):
    """Return mean matrix R (K×M) and second-moment matrices Σ (K×K×M)."""
    R = r_base[:, None] + alpha * Delta_r_m
    Sigma = np.zeros((K, K, M))
    for m in range(M):
        sig_vec = sigma_base + alpha * Delta_sigma_m[:, m]
        diag_sig = np.diag(sig_vec)
        C_alpha = mix_corr(C_base, tilde_C_list[m], alpha, beta=0.5)
        Sigma[:, :, m] = diag_sig @ C_alpha @ diag_sig + np.outer(R[:, m], R[:, m])
    return R, Sigma

# ---------------------------------------------------------------------
# 2. α = 0 の古典 MVP を厳密に解く（和の制約なし、目標リターン制約あり）
# ---------------------------------------------------------------------
R0, Sigma0_arr = build_params(alpha0)

# モデル重みは一様  w_m = 1/M
w_uniform = np.ones(M) / M
# 平均リターンベクトルは、各モデルの期待リターンの平均とする
r_bar     = R0 @ w_uniform

# 共分散行列は、古典設定ではモデル0のデータから計算（元のスクリプトに従う）
Cov_bar   = Sigma0_arr[:,:,0]-np.outer(R0[:,0],R0[:,0])

# ---- 目標リターン mu_tilde を達成し、分散を最小化（和の制約なし）----
# 最適解は pi = lambda * inv(Cov_bar) @ r_bar
# 制約: r_bar^T @ pi = mu_tilde
# r_bar^T @ (lambda * inv(Cov_bar) @ r_bar) = mu_tilde
# lambda * (r_bar^T @ inv(Cov_bar) @ r_bar) = mu_tilde
# lambda = mu_tilde / (r_bar^T @ inv(Cov_bar) @ r_bar)

invC = np.linalg.inv(Cov_bar)

# B_prime = r_bar^T @ invC @ r_bar  (これは元のスクリプトのCに等しい)
B_prime = r_bar @ invC @ r_bar

# lambda を計算 (ただし B_prime がゼロでないことを仮定)
if B_prime < 1e-9: # ゼロに近い場合のエラー回避
     raise ValueError("r_bar @ inv(Cov_bar) @ r_bar is close to zero. Cannot achieve target return.")

lam = mu_tilde / B_prime

# 最適ポートフォリオベクトル (和の制約なし)
pi_star_unconstrained = lam * invC @ r_bar

# このポートフォリオベクトルの期待リターン (mu_tilde )
expected_ret_star = r_bar @ pi_star_unconstrained

# このポートフォリオベクトルの分散
var_star_unconstrained = pi_star_unconstrained @ Cov_bar @ pi_star_unconstrained

# 計算されたポートフォリオの Sharpe Ratio (リスクフリーレート0)
# SR = Expected Return / sqrt(Variance)
# Expected Return は mu_tilde に設定されている
# SR = mu_tilde / sqrt(var_star_unconstrained)
# var_star_unconstrained = (lambda * invC @ r_bar)^T @ Cov_bar @ (lambda * invC @ r_bar)
# = lambda^2 * (invC @ r_bar)^T @ Cov_bar @ (invC @ r_bar)
# = lambda^2 * (r_bar^T @ invC @ Cov_bar @ invC @ r_bar)
# = lambda^2 * (r_bar^T @ invC @ r_bar) = lambda^2 * B_prime
# SR = mu_tilde / sqrt(lambda^2 * B_prime) = mu_tilde / (abs(lambda) * sqrt(B_prime))
# lambda = mu_tilde / B_prime なので
# SR = mu_tilde / (abs(mu_tilde / B_prime) * sqrt(B_prime))
# If mu_tilde > 0, SR = mu_tilde / ((mu_tilde / B_prime) * sqrt(B_prime)) = B_prime / sqrt(B_prime) = sqrt(B_prime)
# If mu_tilde < 0, SR = mu_tilde / ((-mu_tilde / B_prime) * sqrt(B_prime)) = -B_prime / sqrt(B_prime) = -sqrt(B_prime)
# Generally, SR = sign(mu_tilde) * sqrt(B_prime)

SR_calc = np.sign(mu_tilde) * np.sqrt(B_prime) if B_prime > 0 else 0 # B_prime <= 0 の場合はSR定義が問題になる

# ---- λ_min ----
lambda_min_cov = np.min(np.linalg.eigvalsh(Cov_bar))

# ---------------------------------------------------------------------
# 3. CSV 生成 – α-sweep 互換フォーマット
# ---------------------------------------------------------------------
cols_w   = [f"w*_m{m}"   for m in range(M)]
cols_pi  = [f"pi*_k{k}"  for k in range(K)]
cols_ret = [f"cstr_ret_m{m}" for m in range(M)]
cols_lam = [f"lam_m{m}"  for m in range(M)] # ここでの lam は上記の lambda
cols_dpi = [f"dpi*_k{k}" for k in range(K)]

columns = (["alpha", "H_star", "supp_w", "lambda_min",
            "iterations", "SR_rob", "SR_bound"]
           + cols_w + cols_pi + cols_ret + cols_lam + cols_dpi)

# lam_m は robust 問題の各モデルに対するラグランジュ乗数だが、
# ここでは和の制約なしMV問題の単一のラグランジュ乗数 lambda を格納する
# フォーマット互換性のため、lambda を cols_lam の最初の要素に入れる
lam_output = [lam] + [np.nan] * (len(cols_lam) - 1)


row = [alpha0,
       var_star_unconstrained,                    # H_star = 計算されたポートフォリオベクトルの分散
       np.nan,                                  # supp_w   – 固定しないので空欄
       lambda_min_cov,                          # lambda_min of Cov_bar
       np.nan,                                  # iterations – 求解回数無し
       SR_calc,                                 # SR_rob (計算されたポートフォリオのSharpe)
       np.sqrt(B_prime) if B_prime > 0 else np.nan] # SR_bound (理論上限、sqrt(C) または sqrt(B_prime))
row += [np.nan] * len(cols_w)                   # w*  — robust 特有 → NaN
row += list(pi_star_unconstrained)              # pi* 4 つ (和の制約なし、目標リターン達成ベクトル)
# 各モデル m = 0..M-1 における計算されたポートフォリオの期待リターン
row += list(R0.T @ pi_star_unconstrained)       # 各モデル制約リターン
row += lam_output                               # λ   — 単一の lambda を格納
row += [np.nan] * len(cols_dpi)                 # dpi* — NaN

df = pd.DataFrame([row], columns=columns)
df.to_csv("alpha0_classical_mv_unconstrained_target.csv", index=False, float_format="%.10g")
print("CSV written to alpha0_classical_mv_unconstrained_target.csv")

CSV written to alpha0_classical_mv_unconstrained_target.csv


#すべて異なる場合

In [ ]:
# -*- coding: utf-8 -*-
"""
alpha_sweep_full_bounds_reverted_with_dpi.py
Based on alpha_sweep_full_bounds_reverted.py, adds dpi* calculation.
"""

# ... (Imports and Global Constants remain the same) ...
import numpy as np
import pandas as pd
import itertools, logging, sys
from datetime import datetime
import scipy.linalg as la
from scipy.optimize import minimize, Bounds, LinearConstraint

K, M         = 4, 4
mu_tilde     = 0.03
ALPHA_MIN    = 1e-6
ALPHA_MAX    = 1e-5
N_ALPHA      = 101
ALPHA_GRID   = np.linspace(ALPHA_MIN, ALPHA_MAX, N_ALPHA)
MAX_OPT_ITERS = 5000 # Using MAX_OPT_ITERS from reverted code context (was 10000 in original)
tol_grad     = 1e-8
step_size_norm_threshold = 1e-12 # Using threshold from reverted code context
lr_init, lr_max = 0.15, 1.0 # Using lr from reverted code context
INNER_OPT_TOL_FEAS = 1e-9
SOLVER_FEAS_TOL    = 1e-7
GENTOL             = 1e-9
FINITE_DIFF_EPS    = 1e-12

# ---------------------------------------------------------------------
# 1. SYNTHETIC DATA
# ---------------------------------------------------------------------
r_base = np.array([0.02, 0.03, 0.04, 0.05])
Delta_A = np.array([[+1, 1, -1, -1],
                    [+1, 1, -1, -1],
                    [+2, -1, +2, -2],
                    [+2, -2, +2, -2]], dtype=float)
Delta_r_m     = Delta_A / 1000
sigma_base    = np.array([0.20, 0.25, 0.30, 0.35])
Delta_sigma_m = Delta_A / 100

def block_corr(rho_in: float, rho_out: float) -> np.ndarray:
    C = np.full((K, K), rho_out)
    np.fill_diagonal(C, 1.0)
    sectA, sectB = [0, 1], [2, 3]
    for i in sectA:
        for j in sectA:
            if i != j:
                C[i, j] = rho_in
    for i in sectB:
        for j in sectB:
            if i != j:
                C[i, j] = rho_in
    return C

C_base = block_corr(0.75, 0.50)
tilde_C_list = [
    block_corr(0.85, 0.40),
    block_corr(0.65, 0.40),
    block_corr(0.65, 0.60),
    block_corr(0.85, 0.60),
]
beta_corr = 1/ALPHA_MAX


def mix_corr(C0, C1, alpha, beta=0.5, eps=1e-4): # Copied from original context
    C = (1 - beta * alpha) * C0 + beta * alpha * C1
    eigval, eigvec = np.linalg.eigh(C)
    eigval_clip = np.clip(eigval, eps, None)
    C_spd = eigvec @ np.diag(eigval_clip) @ eigvec.T
    D = np.sqrt(np.diag(C_spd))
    # Check for NaN/Inf in D before division
    if np.any(np.isnan(D)) or np.any(np.isinf(D)) or np.any(D <= 1e-10):
        # Fallback or error handling if diagonal elements are non-positive or zero
        # For now, return unnormalized C_spd and indicate issue with lam_min
        return C_spd, -np.inf # Or raise error
    D_inv = np.diag(1.0 / D)
    C_corr = D_inv @ C_spd @ D_inv
    # Ensure diagonal is 1 after potential numerical errors
    np.fill_diagonal(C_corr, 1.0)
    return C_corr, float(np.min(eigval_clip))

def build_params(alpha: float): # Adapted from original robust context
    R_th = r_base[:, None] + alpha * Delta_r_m
    Sigma_th = np.zeros((K, K, M))
    V_th     = np.zeros((K, K, M))
    lambda_min_corr_log = []
    reg_V = 1e-8; reg_S = 1e-8
    for m in range(M):
        sig_vec   = sigma_base + alpha * Delta_sigma_m[:, m]
        diag_sig  = np.diag(sig_vec)
        C_alpha, lam_min = mix_corr(C_base, tilde_C_list[m], alpha, beta=beta_corr, eps=1e-5)
        if lam_min == -np.inf: # Handle mix_corr failure
             raise ValueError(f"mix_corr failed for alpha={alpha}, m={m}")
        lambda_min_corr_log.append(lam_min)
        V_m = diag_sig @ C_alpha @ diag_sig
        min_eig_Vm = np.min(np.linalg.eigvalsh(V_m))
        if min_eig_Vm <= reg_V: V_m += np.eye(K) * (abs(min_eig_Vm) + reg_V)
        V_th[:, :, m] = V_m
        Sigma_th[:, :, m] = V_m + np.outer(R_th[:, m], R_th[:, m])
        min_eig_Sm = np.min(np.linalg.eigvalsh(Sigma_th[:, :, m]))
        if min_eig_Sm <= reg_S: Sigma_th[:, :, m] += np.eye(K) * (abs(min_eig_Sm) + reg_S)
    lambda_min_for_csv = float(min(lambda_min_corr_log)) if lambda_min_corr_log else np.nan
    # Original robust code had a rank check on R_th, might be needed depending on K,M
    if np.linalg.matrix_rank(R_th, tol=1e-12) < M and K >= M:
      raise RuntimeError('R_th is not full column rank')
    return R_th, Sigma_th, V_th, lambda_min_for_csv

# --- UTILITY FUNCTIONS (proj_simplex, Y0, X_piY) ---
# (Assumed identical to previous correct versions, omitted for brevity)
def proj_simplex(v: np.ndarray) -> np.ndarray:
    v = np.asarray(v, float); u = np.sort(v)[::-1]; cssv = np.cumsum(u) - 1
    if (v >= 0).all() and np.isclose(v.sum(), 1.0, atol=1e-9): return v
    try:
        idx_rho = np.where(u * (np.arange(len(u)) + 1) > cssv)[0]
        rho = idx_rho[-1] if len(idx_rho) > 0 else len(u) - 1
        theta = cssv[rho] / (rho + 1); return np.maximum(v - theta, 0.0)
    except IndexError: return np.ones_like(v) / len(v)

def calculate_Y0(R_trial, Sigma_list_trial, mu_tilde_val, k_assets, m_models, rng_y0_pert):
    pi_avg_r = np.mean(R_trial, axis=1)
    if np.linalg.norm(pi_avg_r) > 1e-6: pi_initial_guess = pi_avg_r / np.linalg.norm(pi_avg_r)
    else: pi_initial_guess = np.ones(k_assets) / np.sqrt(k_assets)
    max_tries_feas = 10; found_feasible_pi = False
    for i_try in range(max_tries_feas):
        min_constr_val = np.min(R_trial.T @ pi_initial_guess)
        if min_constr_val >= mu_tilde_val - SOLVER_FEAS_TOL :
             if min_constr_val < mu_tilde_val: pi_initial_guess *= (mu_tilde_val / (min_constr_val if abs(min_constr_val) > 1e-9 else 1e-9)) * 1.01
             found_feasible_pi = True; break
        elif min_constr_val > 0 : pi_initial_guess *= (mu_tilde_val / min_constr_val) * (1.1 + 0.1 * i_try)
        else:
            pi_initial_guess += rng_y0_pert.normal(0, 0.1, size=k_assets)
            if np.linalg.norm(pi_initial_guess) > 1e-6: pi_initial_guess /= np.linalg.norm(pi_initial_guess)
            else: pi_initial_guess = rng_y0_pert.random(k_assets); pi_initial_guess /= (np.linalg.norm(pi_initial_guess) if np.linalg.norm(pi_initial_guess) > 1e-6 else 1)
    if not found_feasible_pi:
        try:
            pi_lstsq = np.linalg.lstsq(R_trial.T, np.full(m_models, mu_tilde_val), rcond=None)[0]
            if np.min(R_trial.T @ pi_lstsq) >= mu_tilde_val - SOLVER_FEAS_TOL: pi_initial_guess = pi_lstsq
            else: pi_initial_guess = np.ones(k_assets) / k_assets
        except np.linalg.LinAlgError: pi_initial_guess = np.ones(k_assets) / k_assets
    t_initial_guess = 0.0
    for m in range(m_models): t_initial_guess = max(t_initial_guess, pi_initial_guess @ Sigma_list_trial[:,:,m] @ pi_initial_guess)
    t_initial_guess = max(1e-6, t_initial_guess)
    x_initial = np.concatenate([pi_initial_guess, [t_initial_guess]])
    func = lambda x: x[-1]; cons = []
    for m in range(m_models): cons.append({'type': 'ineq', 'fun': lambda x, Sm=Sigma_list_trial[:,:,m]: x[-1] - x[:-1] @ Sm @ x[:-1]})
    for m in range(m_models): cons.append({'type': 'ineq', 'fun': lambda x, rm=R_trial[:,m]: x[:-1] @ rm - mu_tilde_val})
    bounds = [(-np.inf, np.inf)] * k_assets + [(1e-8, np.inf)]
    opt_res_Y0 = minimize(func, x_initial, method='SLSQP', bounds=bounds, constraints=cons, options={'ftol': 1e-8, 'disp': False, 'maxiter': 1000})
    pi_Y = None; Y0_val = np.nan
    if opt_res_Y0.success:
        pi_sol, t_sol = opt_res_Y0.x[:-1], opt_res_Y0.x[-1]; feasible = True
        min_ret_check = np.min(R_trial.T @ pi_sol)
        max_quad_check = np.max([pi_sol @ Sigma_list_trial[:,:,m] @ pi_sol for m in range(m_models)])
        if min_ret_check < mu_tilde_val - SOLVER_FEAS_TOL: feasible = False
        if abs(t_sol - max_quad_check) > SOLVER_FEAS_TOL * (1 + abs(t_sol)) + 1e-7 : feasible = False
        if feasible: pi_Y = pi_sol; Y0_val = t_sol
    return pi_Y, Y0_val

def calculate_X_pi_Y(pi_Y, R_trial, Sigma_list_trial, V_list_trial, k_assets, m_models):
    if pi_Y is None: return np.nan
    v_m_vec = np.array([pi_Y @ V_list_trial[:,:,m] @ pi_Y for m in range(m_models)])
    a_m_vec = R_trial.T @ pi_Y
    Q_mat = np.outer(a_m_vec, a_m_vec)
    c_vec = v_m_vec + a_m_vec**2
    obj_func = lambda w: w @ Q_mat @ w - c_vec @ w
    jac_func = lambda w: 2 * Q_mat @ w - c_vec
    bounds_w = Bounds(np.zeros(m_models), np.full(m_models, np.inf))
    constraints_w = LinearConstraint(np.ones((1, m_models)), [1.0], [1.0])
    w_initial = np.ones(m_models) / m_models
    qp_res = minimize(obj_func, w_initial, method='SLSQP', jac=jac_func,
                      bounds=bounds_w, constraints=constraints_w,
                      options={'ftol': 1e-9, 'disp': False})
    if qp_res.success:
        w_opt_for_X = qp_res.x; w_opt_for_X = np.maximum(0, w_opt_for_X); w_opt_for_X /= np.sum(w_opt_for_X)
        var_a_m = w_opt_for_X @ (a_m_vec**2) - (w_opt_for_X @ a_m_vec)**2
        X_piY_val = w_opt_for_X @ v_m_vec + var_a_m
        return max(0, X_piY_val)
    else: return np.nan

# --- INNER MINIMISATION (Using robust version) ---
def inner_opt(w, R, Sigma, lam_tol=-1e-13, rank_scale=1e-10):
    k_assets, m_models = R.shape; rank_tol = rank_scale * np.linalg.norm(R); Ex = R @ w
    Sigma_w_bar = np.sum(np.fromiter((w[m] * Sigma[:, :, m] for m in range(m_models)), dtype=object))
    Vw = Sigma_w_bar - np.outer(Ex, Ex)
    try: min_eig_Vw = np.min(np.linalg.eigvalsh(Vw))
    except np.linalg.LinAlgError: return None, np.inf, Ex, None, None
    if min_eig_Vw <= 1e-10: Vw += np.eye(k_assets) * (abs(min_eig_Vw) + 1e-8)
    try: Vinv = np.linalg.inv(Vw)
    except np.linalg.LinAlgError: return None, np.inf, Ex, None, None
    best_val = np.inf; best_pi = None; best_lam = None; best_subset = None
    for s in range(1, m_models + 1):
        for subset in itertools.combinations(range(m_models), s):
            subset_indices = list(subset); A = R[:, subset_indices].T
            if np.linalg.matrix_rank(A, tol=rank_tol) < s: continue
            Mmat = A @ Vinv @ A.T
            try: cond_Mmat = np.linalg.cond(Mmat)
            except np.linalg.LinAlgError: continue
            if cond_Mmat > 1e8: continue
            try: lam, residuals, rank, s_val = np.linalg.lstsq(Mmat, mu_tilde * np.ones(s), rcond=None)
            except np.linalg.LinAlgError: continue
            if rank < s or (residuals.size > 0 and residuals[0] > 1e-6): continue
            if np.any(lam < lam_tol): continue
            pi = Vinv @ A.T @ lam
            if np.min(R.T @ pi) < mu_tilde - INNER_OPT_TOL_FEAS: continue
            val = pi @ (Sigma_w_bar - np.outer(Ex, Ex)) @ pi # Use original Vw definition for value
            if val < best_val - GENTOL * (1 + abs(best_val)): best_val,best_pi,best_lam,best_subset = val,pi,lam,subset
    if best_pi is None: return None, np.inf, Ex, None, None
    return best_pi, best_val, Ex, best_lam, best_subset

# --- H_and_grad, optimise_outer_reverted (Using original optimisation logic) ---
# Need to copy the original H_and_grad and optimise_outer from the 'reverted' context
# Assuming they are named H_and_grad_reverted and optimise_outer_reverted
# H_and_grad_reverted would call the original inner_opt_reverted
# This requires providing the original inner_opt logic. Let's assume it's available.

# Placeholder for the original inner_opt logic (needs to be provided from the original script)
def inner_opt_reverted(w, R, Sigma, tol_feas=1e-9, lam_tol=-1e-13, rank_scale=1e-10):
    # *** This function needs the exact code from the original alpha_sweep_K4M4_SR.py ***
    # *** For now, using the robust one as a placeholder - REPLACE THIS ***
    # return inner_opt(w, R, Sigma, lam_tol=lam_tol, rank_scale=rank_scale) # Placeholder

    # --- Start of copied original inner_opt logic ---
    rank_tol = rank_scale * np.linalg.norm(R)
    Ex = R @ w
    # Use np.sum with generator directly as in original
    Vw = sum(w[m] * Sigma[:, :, m] for m in range(M)) - np.outer(Ex, Ex)
    try:
        Vinv = np.linalg.inv(Vw)
    except np.linalg.LinAlgError:
        return None, np.inf, Ex, None, None

    best_val = np.inf
    best_pi = best_lam = best_subset = None

    for s in range(1, M + 1):
        for subset in itertools.combinations(range(M), s):
            A = R[:, subset].T
            if np.linalg.matrix_rank(A, tol=rank_tol) < s: continue
            Mmat = A @ Vinv @ A.T
            # Original check might have been slightly different, e.g., no condition number check
            if np.linalg.matrix_rank(Mmat, tol=rank_tol) < s: continue
            try:
                lam = np.linalg.solve(Mmat, mu_tilde * np.ones(s)) # Original used solve
            except np.linalg.LinAlgError:
                 continue # Handle potential solve failure

            if np.any(lam < lam_tol): continue
            pi = Vinv @ A.T @ lam
            # Original check used global tol_feas implicitly? Ensure it's correct.
            if np.any(R.T @ pi < mu_tilde - tol_feas): continue
            val = pi @ Vw @ pi
            # Original comparison tolerance
            if val + 1e-12 < best_val:
                best_val, best_pi, best_lam, best_subset = val, pi, lam, subset

    if best_pi is None:
        return None, np.inf, Ex, None, None
    return best_pi, best_val, Ex, best_lam, best_subset
    # --- End of copied original inner_opt logic ---


def H_and_grad_reverted(w, R, Sigma):
    # Calls the reverted inner_opt
    pi, val, Ex, lam, subset = inner_opt_reverted(w, R, Sigma, tol_feas=INNER_OPT_TOL_FEAS) # Pass tol_feas explicitly
    if val == np.inf or pi is None: # Added pi is None check
        return val, None, pi, lam, subset
    # Grad calculation is standard
    grad = np.array([pi @ Sigma[:, :, m] @ pi - 2 * (pi @ R[:, m]) * (pi @ Ex) for m in range(M)])
    return val, grad, pi, lam, subset

def optimise_outer_reverted(R, Sigma, *, start=None, seed=None):
    # Uses H_and_grad_reverted, rest of logic is from original alpha_sweep_K4M4_SR.py
    if start is None:
        rng = np.random.default_rng(seed) if seed is not None else np.random.default_rng()
        w = proj_simplex(rng.random(M))
    else: w = proj_simplex(start)

    # Initial call using reverted H_and_grad
    val, g, pi, lam, subset = H_and_grad_reverted(w, R, Sigma)
    if g is None: return np.nan, None, None, None, None, 0

    best = (val, w.copy(), pi, lam, subset)
    lr   = lr_init # Use lr_init from original context
    g_prev, w_prev = g.copy(), w.copy()

    # Use MAX_OPT_ITERS from original context (e.g., 10000)
    current_max_opt_iters = 10000 # Match original script context

    for it in range(1, current_max_opt_iters + 1): # Use original MAX_OPT_ITERS
        proj_grad = w - proj_simplex(w - g) # Check KKT condition
        # Use tol_grad from original context
        if np.linalg.norm(proj_grad) < tol_grad: break

        if it > 1:
            dw, dg = w - w_prev, g - g_prev; denom = dg @ dg
            if denom > 1e-12:
                gamma_bb = (dw @ dg) / denom
                if gamma_bb > 0.0:
                    # Use lr_max from original context
                    lr = np.clip(gamma_bb, 1e-4, lr_max)

        lr_curr = lr; accept  = False
        # Line search parameters might differ slightly, use original logic's values/checks
        while lr_curr >= 1e-6: # Original line search lower bound? Assume 1e-6
            w_trial = proj_simplex(w + lr_curr * g)
            # Use step_size_norm_threshold from original context
            if np.linalg.norm(w_trial - w) < step_size_norm_threshold:
                lr_curr = 0.0; break
            # Call reverted H_and_grad
            val_t, g_t, pi_t, lam_t, subset_t = H_and_grad_reverted(w_trial, R, Sigma)

            # Check for failure in H_and_grad_reverted
            if g_t is None:
                 lr_curr *= 0.5; continue

            # Original Armijo check tolerance
            if val_t >= val - 1e-12:
                w_prev, g_prev = w.copy(), g.copy(); w, val, g = w_trial, val_t, g_t
                pi, lam, subset = pi_t, lam_t, subset_t
                # Original best update check (might have used different tolerance)
                if val > best[0]: # Assume simple > check was used originally
                    best = (val, w.copy(), pi, lam, subset)
                # Use original lr update logic
                lr = min(lr_curr * 1.2, lr_max)
                accept = True; break
            lr_curr *= 0.5
        if not accept: break
    return (*best, it) # Return best found solution and iteration count

# --- CALCULATE BOUNDS AND SR ---
def calculate_bounds_and_sr(pi_star, w_star, H_star_val, R_th, Sigma_th, V_th):
    # (Identical to previous robust version)
    SR_rob_val, B_U1_val, B_U2_val = np.nan, np.nan, np.nan
    if pi_star is None or w_star is None or np.isnan(H_star_val): return SR_rob_val, B_U1_val, B_U2_val
    r_bar_ws = R_th @ w_star; numerator_sr = pi_star @ r_bar_ws
    denominator_sr_sq = H_star_val
    if denominator_sr_sq > 1e-12: SR_rob_val = numerator_sr / np.sqrt(denominator_sr_sq)
    elif abs(numerator_sr) < GENTOL: SR_rob_val = 0.0
    Sigma_w_star_bar = np.sum(np.fromiter((w_star[m] * Sigma_th[:, :, m] for m in range(M)), dtype=object))
    Vw_star_check = Sigma_w_star_bar - np.outer(r_bar_ws, r_bar_ws)
    try: Vw_star_inv = np.linalg.inv(Vw_star_check + np.eye(K)*1e-10); s_w_star_num = r_bar_ws @ Vw_star_inv @ r_bar_ws
    except np.linalg.LinAlgError: B_U1_val = np.nan; s_w_star_num=-1 # Assign default for calculation below
    else: # Only calculate if inverse succeeded
        if s_w_star_num < -GENTOL: s_w_star_num = 0
        B_U1_val = np.sqrt(s_w_star_num)
    rho_m_vals = np.zeros(M); all_rho_m_ok = True
    for m in range(M):
        try: Sigma_m_inv = np.linalg.inv(Sigma_th[:,:,m] + np.eye(K)*1e-10); rho_m_vals[m] = R_th[:,m] @ Sigma_m_inv @ R_th[:,m]
        except np.linalg.LinAlgError: all_rho_m_ok = False; break
    if all_rho_m_ok:
        rho_max_val = np.max(rho_m_vals); rho_max_val = min(rho_max_val, 1.0 - 1e-12)
        if rho_max_val < 0: B_U2_val = np.nan
        else: B_U2_val = np.sqrt(rho_max_val / (1.0 - rho_max_val))
    else: B_U2_val = np.nan
    return SR_rob_val, B_U1_val, B_U2_val

# --- FINITE DIFFERENCE HELPER ---
def finite_diff_vec(v_p, v_m, h, length):
    # (Identical to previous)
    if v_p is None or v_m is None or h is None or np.isnan(h) or abs(h) < FINITE_DIFF_EPS: return np.full(length, np.nan)
    # Ensure h is not zero before dividing
    actual_h = h if abs(h) > FINITE_DIFF_EPS else np.sign(h)*FINITE_DIFF_EPS if h!=0 else FINITE_DIFF_EPS
    return (v_p - v_m) / (2 * actual_h) # Use 2*h for central difference

# --- α-SWEEP MAIN FUNCTION (with reverted opt logic and dpi*) ---
def run_alpha_sweep_full_bounds_reverted_with_dpi(csv_path="alpha_sweep_full_bounds_reverted_dpi.csv", *, seed_outer=None):
    log = logging.getLogger("sweep_revert_dpi")
    log.propagate = False
    log.setLevel(logging.INFO)
    if not log.handlers: log.addHandler(logging.StreamHandler(sys.stdout))

    col_w = [f"w*_m{m}" for m in range(M)]; col_pi = [f"pi*_k{k}" for k in range(K)]
    col_ret = [f"cstr_ret_m{m}" for m in range(M)]; col_lam = [f"lam_m{m}" for m in range(M)]
    col_dpi = [f"dpi*_k{k}" for k in range(K)]
    columns = (["alpha", "H_star", "supp_w", "lambda_min_corr", "iterations", "SR_rob",
                "SR_Bound_Sw", "SR_Bound_rho_max", "SR_Bound_Y0", "SR_Bound_XpiY",
                "Y0_val", "XpiY_val"] + col_w + col_pi + col_ret + col_lam + col_dpi)
    rows, cache = [], {}
    rng_y0 = np.random.default_rng(seed_outer + 1 if seed_outer is not None else 43)
    t0 = datetime.now(); y0_fail_count = 0; xpiy_fail_count = 0; opt_fail_count = 0

    log.info(f"========== α-Sweep Full Bounds START (N_ALPHA={N_ALPHA}, Reverted Opt Logic, with dpi*) ==========")

    # --- Define get function for caching ---
    def get(a, seed_for_opt):
        cache_key = (a, seed_for_opt)
        if cache_key not in cache:
            try:
                R_a, Sigma_a, V_a, lam_min_a = build_params(a)
                H_star_a, w_star_a, pi_star_a, lam_star_a, subset_a, iters_a = optimise_outer_reverted(
                    R_a, Sigma_a, seed=seed_for_opt)
                if pi_star_a is None or w_star_a is None or np.isnan(H_star_a): raise ValueError("Optimise_outer_reverted returned None/NaN")
                cache[cache_key] = dict(H_star=H_star_a, w_star=w_star_a, pi_star=pi_star_a,
                                    lam_star=lam_star_a, subset=subset_a, R=R_a, Sigma=Sigma_a, V=V_a,
                                    lambda_min_corr=lam_min_a, iters=iters_a)
            except (RuntimeError, ValueError, np.linalg.LinAlgError, TypeError) as e:
                # log.warning(f"Get failed for alpha={a}. Error: {e}")
                cache[cache_key] = None
        return cache[cache_key]
    # --- End of get function ---

    for idx, alpha in enumerate(ALPHA_GRID):
        if (idx * 100 // N_ALPHA) > ((idx - 1) * 100 // N_ALPHA):
             log.info(f"Progress: {idx * 100 / N_ALPHA:.0f}% (α = {alpha:.5f})")

        current_seed = seed_outer # Use same seed across alpha if provided, else None
        res_c = get(alpha, current_seed)

        if res_c is None:
            opt_fail_count += 1 # Increment failure count here based on get() result
            continue

        H_star, w_star, pi_star, lam_star, subset, R_th, Sigma_th, V_th, lambda_min_val, iters = \
            res_c["H_star"], res_c["w_star"], res_c["pi_star"], res_c["lam_star"], res_c["subset"], \
            res_c["R"], res_c["Sigma"], res_c["V"], res_c["lambda_min_corr"], res_c["iters"]

        SR_rob, B_U1, B_U2 = calculate_bounds_and_sr(pi_star, w_star, H_star, R_th, Sigma_th, V_th)
        pi_Y, Y0_val = calculate_Y0(R_th, Sigma_th, mu_tilde, K, M, rng_y0)
        if pi_Y is None or np.isnan(Y0_val): y0_fail_count += 1
        XpiY_val = calculate_X_pi_Y(pi_Y, R_th, Sigma_th, V_th, K, M)
        if pi_Y is not None and np.isnan(XpiY_val): xpiy_fail_count += 1
        B_L1 = mu_tilde / np.sqrt(Y0_val) if Y0_val is not None and not np.isnan(Y0_val) and Y0_val > 1e-12 else np.nan
        B_L2 = mu_tilde / np.sqrt(XpiY_val) if XpiY_val is not None and not np.isnan(XpiY_val) and XpiY_val > 1e-12 else np.nan

        # --- Calculate dpi* (Finite Difference) using get ---
        dpi_star = np.full(K, np.nan)
        prev_alpha = ALPHA_GRID[idx - 1] if idx > 0 else None
        next_alpha = ALPHA_GRID[idx + 1] if idx < N_ALPHA - 1 else None
        h = np.nan
        pi_p, pi_m = None, None

        if prev_alpha is not None and next_alpha is not None:
            h = min(alpha - prev_alpha, next_alpha - alpha)
            if h < FINITE_DIFF_EPS: h = FINITE_DIFF_EPS # Ensure h is not too small
            res_p = get(alpha + h, current_seed)
            res_m = get(alpha - h, current_seed)
            if res_p is not None: pi_p = res_p["pi_star"]
            if res_m is not None: pi_m = res_m["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / (2*h)

        elif prev_alpha is None and next_alpha is not None: # Forward difference
            h = next_alpha - alpha
            if h >= FINITE_DIFF_EPS:
                res_p = get(next_alpha, current_seed)
                if res_p is not None: pi_p = res_p["pi_star"]
                pi_m = pi_star
                if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / h

        elif next_alpha is None and prev_alpha is not None: # Backward difference
            h = alpha - prev_alpha
            if h >= FINITE_DIFF_EPS:
                res_m = get(prev_alpha, current_seed)
                if res_m is not None: pi_m = res_m["pi_star"]
                pi_p = pi_star
                if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / h
        # Note: Central difference is generally preferred if possible.

        # --- Format other outputs ---
        cstr_ret = R_th.T @ pi_star if pi_star is not None else np.full(M, np.nan)
        lam_vec = np.zeros(M)
        if lam_star is not None and subset is not None:
            try: # Add try-except for safety
                subset_indices = tuple(map(int, subset))
                if len(lam_star) == len(subset_indices):
                    for j, m_idx in enumerate(subset_indices):
                        if 0 <= m_idx < M: lam_vec[m_idx] = lam_star[j]
            except (TypeError, IndexError): # Handle if subset is not iterable or index out of bounds
                pass # Keep lam_vec as zeros

        # --- Append row ---
        row_data = {"alpha": alpha, "H_star": H_star, "supp_w": int((w_star > 1e-6).sum()),
                    "lambda_min_corr": lambda_min_val, "iterations": iters, "SR_rob": SR_rob,
                    "SR_Bound_Sw": B_U1, "SR_Bound_rho_max": B_U2, "SR_Bound_Y0": B_L1, "SR_Bound_XpiY": B_L2,
                    "Y0_val": Y0_val, "XpiY_val": XpiY_val}
        row_data.update({f"w*_m{m}": w_star[m] for m in range(M)})
        row_data.update({f"pi*_k{k}": pi_star[k] for k in range(K)})
        row_data.update({f"cstr_ret_m{m}": cstr_ret[m] for m in range(M)})
        row_data.update({f"lam_m{m}": lam_vec[m] for m in range(M)})
        row_data.update({f"dpi*_k{k}": dpi_star[k] for k in range(K)}) # Add calculated dpi_star
        rows.append(row_data)

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(csv_path, index=False, float_format="%.10g")
    log.info(f"CSV written to {csv_path}  (elapsed {(datetime.now() - t0).total_seconds():.1f}s)")
    log.info(f"Total Opt Fails: {opt_fail_count}, Y0 Fails: {y0_fail_count}, XpiY Fails: {xpiy_fail_count}")
    return df

# --- Need to define optimise_outer_reverted and H_and_grad_reverted using original logic ---
# Assuming these are defined elsewhere in the execution environment based on the original script.
# If not, they need to be copied/pasted here.

# --- MAIN EXECUTION ---
if __name__ == "__main__":
     # Make sure optimise_outer_reverted and H_and_grad_reverted are defined correctly based on the original script
     # that produced the successful results before adding bounds.
     # Example call:
     run_alpha_sweep_full_bounds_reverted_with_dpi(seed_outer=123)

========== α-Sweep Full Bounds START (N_ALPHA=101, Reverted Opt Logic, with dpi*) ==========
Progress: 0% (α = 0.00000)
Progress: 2% (α = 0.00000)
Progress: 3% (α = 0.00000)
Progress: 4% (α = 0.00000)
Progress: 5% (α = 0.00000)
Progress: 6% (α = 0.00000)
Progress: 7% (α = 0.00000)
Progress: 8% (α = 0.00000)
Progress: 9% (α = 0.00000)
Progress: 10% (α = 0.00000)
Progress: 11% (α = 0.00000)
Progress: 12% (α = 0.00000)
Progress: 13% (α = 0.00000)
Progress: 14% (α = 0.00000)
Progress: 15% (α = 0.00000)
Progress: 16% (α = 0.00000)
Progress: 17% (α = 0.00000)
Progress: 18% (α = 0.00000)
Progress: 19% (α = 0.00000)
Progress: 20% (α = 0.00000)


#$r^m$のみが異なる場合

In [ ]:
# -*- coding: utf-8 -*-
"""
halfpair_sweep_exact_qp_bounds_dpi.py
- Robust MVP (V fixed, r varies) α-sweep using EXACT inner QP solve.
- Calculates bounds from Thm 2, Thm 6, Thm 8 and dpi*.
"""

# ---------------------------------------------------------------------
# 0. IMPORTS & CONSTANTS
# ---------------------------------------------------------------------
import numpy as np
import itertools, math, logging, sys
import scipy.linalg as la
import pandas as pd
from datetime import datetime
from scipy.optimize import minimize, Bounds, LinearConstraint

K, M         = 4, 4
mu_tilde     = 0.03


INNER_OPT_TOL_FEAS = 1e-9
SOLVER_FEAS_TOL    = 1e-7
GENTOL             = 1e-9
FINITE_DIFF_EPS    = 1e-12

# ---------------------------------------------------------------------
# 1. SYNTHETIC DATA & FIXED PARAMETERS
# ---------------------------------------------------------------------
r_base = np.array([0.02, 0.03, 0.04, 0.05])
Delta_A = np.array([[+1, 1, -1, -1],[+1, 1, -1, -1],
                    [+2, -1, +2, -2],[+2, -2, +2, -2]], dtype=float)
Delta_r_m = Delta_A / 1000
sigma_base = np.array([0.20, 0.25, 0.30, 0.35])

def block_corr(rho_in: float, rho_out: float) -> np.ndarray:
    C = np.full((K, K), rho_out); np.fill_diagonal(C, 1.0)
    sectA, sectB = [0, 1], [2, 3];
    for i in sectA:
        for j in sectA:
            if i != j: C[i, j] = rho_in
    for i in sectB:
        for j in sectB:
            if i != j: C[i, j] = rho_in
    return C

C_base = block_corr(0.75, 0.50)
Vbar   = np.diag(sigma_base) @ C_base @ np.diag(sigma_base)
min_eig_Vbar = np.min(np.linalg.eigvalsh(Vbar))
if min_eig_Vbar <= 1e-9: Vbar += np.eye(K) * (abs(min_eig_Vbar) + 1e-8)
lambda_min_Vbar = float(np.min(np.linalg.eigvalsh(Vbar)))

def get_params_r_only(alpha):
    R_th = r_base[:, None] + alpha * Delta_r_m
    Sigma_th = np.zeros((K, K, M))
    V_th = np.repeat(Vbar[:, :, np.newaxis], M, axis=2) # V is fixed
    for m in range(M):
         Sigma_th[:, :, m] = Vbar + np.outer(R_th[:, m], R_th[:, m])
         min_eig_Sm = np.min(np.linalg.eigvalsh(Sigma_th[:, :, m]))
         if min_eig_Sm <= 1e-9: Sigma_th[:, :, m] += np.eye(K) * (abs(min_eig_Sm) + 1e-8)
    return R_th, Sigma_th, V_th

# ---------------------------------------------------------------------
# 2. UTILITY FUNCTIONS (proj_simplex, Y0, X_piY)
# ---------------------------------------------------------------------
# (Identical to previous correct versions, omitted for brevity)
def proj_simplex(v: np.ndarray) -> np.ndarray:
    v = np.asarray(v, float); u = np.sort(v)[::-1]; cssv = np.cumsum(u) - 1
    if (v >= 0).all() and np.isclose(v.sum(), 1.0, atol=1e-9): return v
    try:
        idx_rho = np.where(u * (np.arange(len(u)) + 1) > cssv)[0]
        rho = idx_rho[-1] if len(idx_rho) > 0 else len(u) - 1
        theta = cssv[rho] / (rho + 1); return np.maximum(v - theta, 0.0)
    except IndexError: return np.ones_like(v) / len(v)

def calculate_Y0(R_trial, Sigma_list_trial, mu_tilde_val, k_assets, m_models, rng_y0_pert):
    pi_avg_r = np.mean(R_trial, axis=1)
    if np.linalg.norm(pi_avg_r) > 1e-6: pi_initial_guess = pi_avg_r / np.linalg.norm(pi_avg_r)
    else: pi_initial_guess = np.ones(k_assets) / np.sqrt(k_assets)
    max_tries_feas = 10; found_feasible_pi = False
    for i_try in range(max_tries_feas):
        min_constr_val = np.min(R_trial.T @ pi_initial_guess)
        if min_constr_val >= mu_tilde_val - SOLVER_FEAS_TOL :
             if min_constr_val < mu_tilde_val: pi_initial_guess *= (mu_tilde_val / (min_constr_val if abs(min_constr_val) > 1e-9 else 1e-9)) * 1.01
             found_feasible_pi = True; break
        elif min_constr_val > 0 : pi_initial_guess *= (mu_tilde_val / min_constr_val) * (1.1 + 0.1 * i_try)
        else:
            pi_initial_guess += rng_y0_pert.normal(0, 0.1, size=k_assets)
            if np.linalg.norm(pi_initial_guess) > 1e-6: pi_initial_guess /= np.linalg.norm(pi_initial_guess)
            else: pi_initial_guess = rng_y0_pert.random(k_assets); pi_initial_guess /= (np.linalg.norm(pi_initial_guess) if np.linalg.norm(pi_initial_guess) > 1e-6 else 1)
    if not found_feasible_pi:
        try:
            pi_lstsq = np.linalg.lstsq(R_trial.T, np.full(m_models, mu_tilde_val), rcond=None)[0]
            if np.min(R_trial.T @ pi_lstsq) >= mu_tilde_val - SOLVER_FEAS_TOL: pi_initial_guess = pi_lstsq
            else: pi_initial_guess = np.ones(k_assets) / k_assets
        except np.linalg.LinAlgError: pi_initial_guess = np.ones(k_assets) / k_assets
    t_initial_guess = 0.0
    for m in range(m_models): t_initial_guess = max(t_initial_guess, pi_initial_guess @ Sigma_list_trial[:,:,m] @ pi_initial_guess)
    t_initial_guess = max(1e-6, t_initial_guess)
    x_initial = np.concatenate([pi_initial_guess, [t_initial_guess]])
    func = lambda x: x[-1]; cons = []
    for m in range(m_models): cons.append({'type': 'ineq', 'fun': lambda x, Sm=Sigma_list_trial[:,:,m]: x[-1] - x[:-1] @ Sm @ x[:-1]})
    for m in range(m_models): cons.append({'type': 'ineq', 'fun': lambda x, rm=R_trial[:,m]: x[:-1] @ rm - mu_tilde_val})
    bounds = [(-np.inf, np.inf)] * k_assets + [(1e-8, np.inf)]
    opt_res_Y0 = minimize(func, x_initial, method='SLSQP', bounds=bounds, constraints=cons, options={'ftol': 1e-8, 'disp': False, 'maxiter': 1000})
    pi_Y = None; Y0_val = np.nan
    if opt_res_Y0.success:
        pi_sol, t_sol = opt_res_Y0.x[:-1], opt_res_Y0.x[-1]; feasible = True
        min_ret_check = np.min(R_trial.T @ pi_sol)
        max_quad_check = np.max([pi_sol @ Sigma_list_trial[:,:,m] @ pi_sol for m in range(m_models)])
        if min_ret_check < mu_tilde_val - SOLVER_FEAS_TOL: feasible = False
        if abs(t_sol - max_quad_check) > SOLVER_FEAS_TOL * (1 + abs(t_sol)) + 1e-7 : feasible = False
        if feasible: pi_Y = pi_sol; Y0_val = t_sol
    return pi_Y, Y0_val

def calculate_X_pi_Y(pi_Y, R_trial, Sigma_list_trial, V_list_trial, k_assets, m_models):
    if pi_Y is None: return np.nan
    v_m_vec = np.array([pi_Y @ V_list_trial[:,:,m] @ pi_Y for m in range(m_models)])
    a_m_vec = R_trial.T @ pi_Y
    Q_mat = np.outer(a_m_vec, a_m_vec)
    c_vec = v_m_vec + a_m_vec**2
    obj_func = lambda w: w @ Q_mat @ w - c_vec @ w
    jac_func = lambda w: 2 * Q_mat @ w - c_vec
    bounds_w = Bounds(np.zeros(m_models), np.full(m_models, np.inf))
    constraints_w = LinearConstraint(np.ones((1, m_models)), [1.0], [1.0])
    w_initial = np.ones(m_models) / m_models
    qp_res = minimize(obj_func, w_initial, method='SLSQP', jac=jac_func,
                      bounds=bounds_w, constraints=constraints_w,
                      options={'ftol': 1e-9, 'disp': False})
    if qp_res.success:
        w_opt_for_X = qp_res.x; w_opt_for_X = np.maximum(0, w_opt_for_X); w_opt_for_X /= np.sum(w_opt_for_X)
        var_a_m = w_opt_for_X @ (a_m_vec**2) - (w_opt_for_X @ a_m_vec)**2
        X_piY_val = w_opt_for_X @ v_m_vec + var_a_m # V_m is constant Vbar here
        return max(0, X_piY_val)
    else: return np.nan

# ---------------------------------------------------------------------
# 3. INNER QP SOLVER (Replaces inner_halfpair)
# ---------------------------------------------------------------------
def solve_inner_qp(w_ij, R_alpha, Vbar_fixed, mu_tilde_val, k_assets, m_models):
    """Solves min_{pi in A(alpha)} pi^T V^w_ij pi"""
    i, j = np.where(w_ij > 1e-6)[0] # Get indices i, j from w_ij
    r_i, r_j = R_alpha[:, i], R_alpha[:, j]

    # Calculate V^w_ij = Vbar + Cov_w_ij[r^m]
    # Cov_w_ij[r^m] = 0.5*r_i*r_i^T + 0.5*r_j*r_j^T - (0.5*r_i+0.5*r_j)(0.5*r_i+0.5*r_j)^T
    r_bar_ij = 0.5 * (r_i + r_j)
    cov_r_ij = 0.5 * (np.outer(r_i, r_i) + np.outer(r_j, r_j)) - np.outer(r_bar_ij, r_bar_ij)
    Vw_ij = Vbar_fixed + cov_r_ij

    # Ensure Vw_ij is SPD for the QP solver
    try:
        min_eig_Vw_ij = np.min(np.linalg.eigvalsh(Vw_ij))
        if min_eig_Vw_ij <= 1e-10: Vw_ij += np.eye(k_assets) * (abs(min_eig_Vw_ij) + 1e-8)
    except np.linalg.LinAlgError: return np.nan, None # Cannot solve if Vw is problematic

    # Objective: minimize 0.5 * pi^T (2 * Vw_ij) pi
    P_qp = 2 * Vw_ij # Factor of 2 needed for standard QP form with 1/2
    q_qp = np.zeros(k_assets) # No linear term in objective

    # Constraints: A_ub @ pi <= b_ub  =>  -R_alpha^T @ pi <= -mu_tilde_val
    A_ub = -R_alpha.T
    b_ub = -np.full(m_models, mu_tilde_val)

    # Initial guess (e.g., from previous Y0 calculation or simple)
    pi_initial_guess = np.linalg.lstsq(R_alpha.T, np.full(m_models, mu_tilde_val), rcond=None)[0]
    # Ensure initial guess is somewhat feasible for constraints
    min_ret_init = np.min(R_alpha.T @ pi_initial_guess)
    if min_ret_init < mu_tilde_val - INNER_OPT_TOL_FEAS:
         # If lstsq doesn't work, try scaling equi-weight
         pi_eq = np.ones(k_assets) / k_assets
         min_ret_eq = np.min(R_alpha.T @ pi_eq)
         if min_ret_eq > 1e-9 : # Avoid division by zero/small
             pi_initial_guess = pi_eq * (mu_tilde_val / min_ret_eq) * 1.01 # Scale up
         else: # Fallback if equi-weight also fails
             pi_initial_guess = pi_eq # Use unscaled

    # Define objective and constraints for scipy.optimize.minimize
    obj_func_qp = lambda pi: 0.5 * pi @ P_qp @ pi
    jac_func_qp = lambda pi: P_qp @ pi # Gradient = P_qp * pi

    # Constraints Ax >= b => -Ax <= -b
    constraints_qp = [{'type': 'ineq', 'fun': lambda pi, m=m: pi @ R_alpha[:,m] - mu_tilde_val} for m in range(m_models)]

    # Solve the QP
    qp_result = minimize(obj_func_qp, pi_initial_guess, method='SLSQP',
                         jac=jac_func_qp, constraints=constraints_qp,
                         options={'ftol': 1e-9, 'disp': False})

    if qp_result.success:
        pi_sol = qp_result.x
        # Verify feasibility of solution strictly
        if np.min(R_alpha.T @ pi_sol) >= mu_tilde_val - SOLVER_FEAS_TOL:
            # Calculate the true objective value H_ij = pi_sol^T Vw_ij pi_sol
            H_ij_val = pi_sol @ Vw_ij @ pi_sol
            return H_ij_val, pi_sol
        else:
            # print(f"Warning: QP solution for pair ({i},{j}) infeasible. Min ret: {np.min(R_alpha.T @ pi_sol)}")
            return np.nan, None
    else:
        # print(f"Warning: Inner QP solver failed for pair ({i},{j}). Status: {qp_result.status}, Msg: {qp_result.message}")
        return np.nan, None

# ---------------------------------------------------------------------
# 4. CALCULATE BOUNDS and SR_rob (for exact ½–½ case)
# ---------------------------------------------------------------------
def calculate_bounds_and_sr_exact_halfpair(pi_star, w_star, H_star_val, R_alpha, Sigma_alpha, Vbar_fixed):
    # This function now assumes H_star_val is the true max_ij H_ij^* = pi_star^T V^w_star pi_star
    SR_rob_val, B_U1_val, B_U2_val = np.nan, np.nan, np.nan
    if pi_star is None or w_star is None or np.isnan(H_star_val): return SR_rob_val, B_U1_val, B_U2_val

    # SR_rob: Use the H_star_val directly as denominator^2
    r_bar_ws = R_alpha @ w_star; numerator_sr = pi_star @ r_bar_ws
    denominator_sr_sq = H_star_val # H_star_val IS pi* Vw* pi*
    if denominator_sr_sq > 1e-12: SR_rob_val = numerator_sr / np.sqrt(denominator_sr_sq)
    elif abs(numerator_sr) < GENTOL: SR_rob_val = 0.0

    # B_U1 = sqrt(S(w*))
    # Calculate Vw* again (or pass it?)
    Sigma_w_star_bar = np.sum(np.fromiter((w_star[m] * Sigma_alpha[:, :, m] for m in range(M)), dtype=object))
    Vw_star = Sigma_w_star_bar - np.outer(r_bar_ws, r_bar_ws)
    try:
        Vw_star_inv = np.linalg.inv(Vw_star + np.eye(K)*1e-10); s_w_star_num = r_bar_ws @ Vw_star_inv @ r_bar_ws
        if s_w_star_num < -GENTOL: s_w_star_num = 0; B_U1_val = np.sqrt(s_w_star_num)
    except np.linalg.LinAlgError: B_U1_val = np.nan

    # B_U2 = sqrt(rho_max / (1-rho_max))
    rho_m_vals = np.zeros(M); all_rho_m_ok = True
    for m in range(M):
        try: Sigma_m_inv = np.linalg.inv(Sigma_alpha[:,:,m] + np.eye(K)*1e-10); rho_m_vals[m] = R_alpha[:,m] @ Sigma_m_inv @ R_alpha[:,m]
        except np.linalg.LinAlgError: all_rho_m_ok = False; break
    if all_rho_m_ok:
        rho_max_val = np.max(rho_m_vals); rho_max_val = min(rho_max_val, 1.0 - 1e-12)
        if rho_max_val < 0: B_U2_val = np.nan
        else: B_U2_val = np.sqrt(rho_max_val / (1.0 - rho_max_val))
    else: B_U2_val = np.nan
    return SR_rob_val, B_U1_val, B_U2_val

# ---------------------------------------------------------------------
# 5. FINITE DIFFERENCE HELPER
# ---------------------------------------------------------------------
def finite_diff_vec(v_p, v_m, h, length):
    if v_p is None or v_m is None or h is None or np.isnan(h) or abs(h) < FINITE_DIFF_EPS: return np.full(length, np.nan)
    actual_h = h if abs(h) > FINITE_DIFF_EPS else np.sign(h)*FINITE_DIFF_EPS if h!=0 else FINITE_DIFF_EPS
    return (v_p - v_m) / (2 * actual_h)

# ---------------------------------------------------------------------
# 6. α-SWEEP MAIN FUNCTION (Exact QP for ½–½ case)
# ---------------------------------------------------------------------
def run_alpha_sweep_halfpair_exact_qp(csv_path="halfpair_exact_qp_bounds_dpi.csv"):
    log = logging.getLogger("sweep_halfpair_exact")
    log.propagate = False
    log.setLevel(logging.INFO)
    if not log.handlers: log.addHandler(logging.StreamHandler(sys.stdout))
    log.info("========== ½–½ Exact QP α-Sweep Full Bounds with dpi* START ==========")

    col_w   = [f"w*_m{m}"   for m in range(M)]
    col_pi  = [f"pi*_k{k}"  for k in range(K)]
    col_ret = [f"cstr_ret_m{m}" for m in range(M)]
    col_lam = [f"lam_m{m}"  for m in range(M)] # Note: QP doesn't directly give lambda_m for robust problem
    col_dpi = [f"dpi*_k{k}" for k in range(K)]
    columns = (["alpha", "H_star", "supp_w", "lambda_min_Vbar", "iterations", # iterations not applicable
                "SR_rob", "SR_Bound_Sw", "SR_Bound_rho_max",
                "SR_Bound_Y0", "SR_Bound_XpiY",
                "Y0_val", "XpiY_val"]
               + col_w + col_pi + col_ret + col_lam + col_dpi)

    rows, cache = [], {}      # cache[alpha] = dict(pi_star, w_star, H_star)
    rng_y0 = np.random.default_rng(12345)
    t0 = datetime.now()
    y0_fail_count = 0; xpiy_fail_count = 0; inner_qp_fail_count = 0

    # --- Define get helper for caching exact halfpair results ---
    def get_exact_halfpair(alpha_val):
        cache_key = alpha_val
        if cache_key not in cache:
            try:
                R_a, Sigma_a, V_a = get_params_r_only(alpha_val)
                best_H_star_a = -math.inf; best_result_a = None; num_qp_fails = 0

                for i, j in itertools.combinations(range(M), 2):
                    w_ij = np.zeros(M); w_ij[[i,j]] = 0.5
                    H_ij_val, pi_ij = solve_inner_qp(w_ij, R_a, Vbar, mu_tilde, K, M)

                    if pi_ij is None or np.isnan(H_ij_val):
                        num_qp_fails += 1; continue # Count failure but continue

                    if H_ij_val > best_H_star_a:
                        best_H_star_a = H_ij_val
                        # Store necessary info associated with the best pair found so far
                        best_result_a = {"H_star": H_ij_val, "pi_star": pi_ij, "w_star": w_ij,
                                         "R": R_a, "Sigma": Sigma_a, "V": V_a} # Keep V_a (copies of Vbar)

                if best_result_a is None: # If all pairs failed
                    raise ValueError(f"All inner QPs failed for alpha={alpha_val}")

                cache[cache_key] = best_result_a
                # Log if some QPs failed but an overall best was found
                # if num_qp_fails > 0: log.debug(f"Alpha {alpha_val:.3f}: {num_qp_fails} inner QPs failed.")

            except Exception as e:
                # log.warning(f"Get_exact_halfpair failed for alpha={alpha_val}. Error: {e}")
                cache[cache_key] = None
        return cache[cache_key]
    # --- End of get helper ---

    for idx, alpha in enumerate(ALPHA_GRID):
        if (idx * 100 // N_ALPHA) > ((idx - 1) * 100 // N_ALPHA):
             log.info(f"Progress: {idx * 100 / N_ALPHA:.0f}% (α = {alpha:.5f})")

        res_c = get_exact_halfpair(alpha)

        if res_c is None:
            inner_qp_fail_count += 1 # Increment failure count based on get helper result
            continue

        H_star  = res_c["H_star"]; pi_star = res_c["pi_star"]; w_star = res_c["w_star"]
        R_th = res_c["R"]; Sigma_th = res_c["Sigma"]; V_th = res_c["V"] # V_th contains copies of Vbar

        SR_rob, B_U1, B_U2 = calculate_bounds_and_sr_exact_halfpair(pi_star, w_star, H_star, R_th, Sigma_th, Vbar)
        pi_Y, Y0_val = calculate_Y0(R_th, Sigma_th, mu_tilde, K, M, rng_y0)
        if pi_Y is None or np.isnan(Y0_val): y0_fail_count += 1
        XpiY_val = calculate_X_pi_Y(pi_Y, R_th, Sigma_th, V_th, K, M)
        if pi_Y is not None and np.isnan(XpiY_val): xpiy_fail_count += 1
        B_L1 = mu_tilde / np.sqrt(Y0_val) if Y0_val is not None and not np.isnan(Y0_val) and Y0_val > 1e-12 else np.nan
        B_L2 = mu_tilde / np.sqrt(XpiY_val) if XpiY_val is not None and not np.isnan(XpiY_val) and XpiY_val > 1e-12 else np.nan

        # --- Calculate dpi* (Finite Difference) using get ---
        dpi_star = np.full(K, np.nan)
        prev_alpha = ALPHA_GRID[idx - 1] if idx > 0 else None
        next_alpha = ALPHA_GRID[idx + 1] if idx < N_ALPHA - 1 else None
        h = np.nan; pi_p, pi_m = None, None
        if prev_alpha is not None and next_alpha is not None:
            h = min(alpha - prev_alpha, next_alpha - alpha)
            if h < FINITE_DIFF_EPS: h = FINITE_DIFF_EPS
            res_p = get_exact_halfpair(alpha + h)
            res_m = get_exact_halfpair(alpha - h)
            if res_p is not None: pi_p = res_p["pi_star"]
            if res_m is not None: pi_m = res_m["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / (2*h)
        elif prev_alpha is None and next_alpha is not None:
            h = next_alpha - alpha
            if h >= FINITE_DIFF_EPS: res_p = get_exact_halfpair(next_alpha); pi_m = pi_star
            if res_p is not None: pi_p = res_p["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / h
        elif next_alpha is None and prev_alpha is not None:
            h = alpha - prev_alpha
            if h >= FINITE_DIFF_EPS: res_m = get_exact_halfpair(prev_alpha); pi_p = pi_star
            if res_m is not None: pi_m = res_m["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / h

        # --- Format other outputs ---
        cstr_ret = R_th.T @ pi_star if pi_star is not None else np.full(M, np.nan)
        # Lagrange multipliers for the robust problem are not directly computed here
        lam_vec = np.full(M, np.nan)

        # --- Append row ---
        row_data = {"alpha": alpha, "H_star": H_star, "supp_w": 2, "lambda_min_Vbar": lambda_min_Vbar,
                    "iterations": np.nan, "SR_rob": SR_rob, "SR_Bound_Sw": B_U1, "SR_Bound_rho_max": B_U2,
                    "SR_Bound_Y0": B_L1, "SR_Bound_XpiY": B_L2, "Y0_val": Y0_val, "XpiY_val": XpiY_val}
        row_data.update({f"w*_m{m}": w_star[m] for m in range(M)})
        row_data.update({f"pi*_k{k}": pi_star[k] for k in range(K)})
        row_data.update({f"cstr_ret_m{m}": cstr_ret[m] for m in range(M)})
        row_data.update({f"lam_m{m}": lam_vec[m] for m in range(M)}) # Placeholder NaN
        row_data.update({f"dpi*_k{k}": dpi_star[k] for k in range(K)})
        rows.append(row_data)

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(csv_path, index=False, float_format="%.10g")
    log.info(f"CSV written to {csv_path}  (elapsed {(datetime.now() - t0).total_seconds():.1f}s)")
    log.info(f"Total Inner QP Fails (at least one pair failed): {inner_qp_fail_count}, Y0 Fails: {y0_fail_count}, XpiY Fails: {xpiy_fail_count}")
    return df

# ---------------------------------------------------------------------
# 7. MAIN EXECUTION
# ---------------------------------------------------------------------
if __name__ == "__main__":
    run_alpha_sweep_halfpair_exact_qp()

#$sigma^m$のみが異なる場合

In [ ]:
# -*- coding: utf-8 -*-
"""
sigma_only_sweep_full_bounds_dpi.py
- Robust MVP (r, C fixed, sigma varies) α-sweep.
- Calculates bounds from Thm 2 (S(w*)), Thm 6 (Y0), Thm 8 (X_piY).
- Includes dpi* calculation.
Based on the provided sigma_only script.
"""
import numpy as np
import pandas as pd
import scipy.linalg as la
from datetime import datetime
from scipy.optimize import minimize, Bounds, LinearConstraint # For Y0, X_piY
import logging, sys

# ... (Global constants and other functions remain the same) ...
K, M = 4, 4
mu_tilde = 0.03

ALPHA_GRID   = np.linspace(ALPHA_MIN, ALPHA_MAX, N_ALPHA)
MAX_OPT_ITERS = 1000
tol_grad = 1e-8
step_size_norm_threshold = 1e-10
INNER_OPT_TOL_FEAS = 1e-9
SOLVER_FEAS_TOL    = 1e-7
GENTOL             = 1e-9
FINITE_DIFF_EPS    = 1e-12
r_bar = np.array([0.02, 0.03, 0.04, 0.05])
sigma_base = np.array([0.20, 0.25, 0.30, 0.35])
Delta_A =np.array([[+1, 1, -1, -1],[+1, 1, -1, -1],
                   [+2, -1, +2, -2],[+2, -2, +2, -2]], dtype=float)
Delta_sigma_m = Delta_A / 100
def block_corr(rho_in, rho_out):
    C = np.full((K, K), rho_out); np.fill_diagonal(C, 1.0)
    for i, j in ((0, 1), (2, 3)): C[i, j] = C[j, i] = rho_in
    return C
C_base = block_corr(0.75, 0.50)
def proj_simplex(v):
    v = np.asarray(v, float)
    if (v >= 0).all() and np.isclose(v.sum(), 1.0, atol=1e-10): return v
    u = np.sort(v)[::-1]; cssv = np.cumsum(u) - 1
    try:
        idx_rho = np.where(u * (np.arange(len(u)) + 1) > cssv)[0]
        rho = idx_rho[-1] if len(idx_rho) > 0 else len(u) - 1
        theta = cssv[rho] / (rho + 1); return np.maximum(v - theta, 0.0)
    except IndexError: return np.ones_like(v) / len(v)
def build_params_sigma_only(alpha):
    R_th = np.tile(r_bar[:, np.newaxis], (1, M))
    Sigma_th = np.zeros((K, K, M)); V_th = np.zeros((K, K, M))
    lambda_min_V_log = []; reg_V = 1e-8; reg_S = 1e-8
    for m in range(M):
        σ = sigma_base + alpha * Delta_sigma_m[:, m]
        V_m = np.diag(σ) @ C_base @ np.diag(σ)
        min_eig_Vm = np.min(np.linalg.eigvalsh(V_m))
        if min_eig_Vm <= reg_V: V_m += np.eye(K) * (abs(min_eig_Vm) + reg_V)
        V_th[:, :, m] = V_m
        lambda_min_V_log.append(np.min(np.linalg.eigvalsh(V_m)))
        Sigma_th[:, :, m] = V_m + np.outer(r_bar, r_bar)
        min_eig_Sm = np.min(np.linalg.eigvalsh(Sigma_th[:, :, m]))
        if min_eig_Sm <= reg_S: Sigma_th[:, :, m] += np.eye(K) * (abs(min_eig_Sm) + reg_S)
    lambda_min_for_csv = float(min(lambda_min_V_log)) if lambda_min_V_log else np.nan
    return R_th, Sigma_th, V_th, lambda_min_for_csv
def calculate_Y0(R_trial, Sigma_list_trial, mu_tilde_val, k_assets, m_models, rng_y0_pert):
    pi_avg_r = np.mean(R_trial, axis=1); k_assets=R_trial.shape[0]; m_models=R_trial.shape[1]
    if np.linalg.norm(pi_avg_r) > 1e-6: pi_initial_guess = pi_avg_r / np.linalg.norm(pi_avg_r)
    else: pi_initial_guess = np.ones(k_assets) / np.sqrt(k_assets)
    max_tries_feas = 10; found_feasible_pi = False; SOLVER_FEAS_TOL=1e-7
    constraint_func = lambda pi: pi @ R_trial[:, 0] - mu_tilde_val
    for i_try in range(max_tries_feas):
        if constraint_func(pi_initial_guess) >= -SOLVER_FEAS_TOL : found_feasible_pi = True; break
        else: current_ret = pi_initial_guess @ R_trial[:, 0]
        if current_ret > 1e-9: pi_initial_guess *= (mu_tilde_val / current_ret) * (1.01 + 0.1*i_try)
        else: pi_initial_guess += rng_y0_pert.normal(0, 0.1, size=k_assets); norm_pi = np.linalg.norm(pi_initial_guess); pi_initial_guess /= norm_pi if norm_pi > 1e-6 else 1
    if not found_feasible_pi: pi_initial_guess = np.ones(k_assets) / k_assets
    t_initial_guess = 0.0
    for m in range(m_models): t_initial_guess = max(t_initial_guess, pi_initial_guess @ Sigma_list_trial[:,:,m] @ pi_initial_guess)
    t_initial_guess = max(1e-6, t_initial_guess)
    x_initial = np.concatenate([pi_initial_guess, [t_initial_guess]])
    func = lambda x: x[-1]; cons = []
    for m in range(m_models): cons.append({'type': 'ineq', 'fun': lambda x, Sm=Sigma_list_trial[:,:,m]: x[-1] - x[:-1] @ Sm @ x[:-1]})
    cons.append({'type': 'ineq', 'fun': lambda x, rm=R_trial[:,0]: x[:-1] @ rm - mu_tilde_val})
    bounds = [(-np.inf, np.inf)] * k_assets + [(1e-8, np.inf)]
    opt_res_Y0 = minimize(func, x_initial, method='SLSQP', bounds=bounds, constraints=cons, options={'ftol': 1e-8, 'disp': False, 'maxiter': 1000})
    pi_Y = None; Y0_val = np.nan
    if opt_res_Y0.success:
        pi_sol, t_sol = opt_res_Y0.x[:-1], opt_res_Y0.x[-1]; feasible = True
        min_ret_check = pi_sol @ R_trial[:,0]
        max_quad_check = np.max([pi_sol @ Sigma_list_trial[:,:,m] @ pi_sol for m in range(m_models)])
        if min_ret_check < mu_tilde_val - SOLVER_FEAS_TOL: feasible = False
        if abs(t_sol - max_quad_check) > SOLVER_FEAS_TOL * (1 + abs(t_sol)) + 1e-7 : feasible = False
        if feasible: pi_Y = pi_sol; Y0_val = t_sol
    return pi_Y, Y0_val
def calculate_X_pi_Y(pi_Y, R_trial, Sigma_list_trial, V_list_trial, k_assets, m_models):
    if pi_Y is None: return np.nan
    v_m_vec = np.array([pi_Y @ V_list_trial[:,:,m] @ pi_Y for m in range(m_models)])
    a_m_vec = R_trial[:,0].T @ pi_Y; a_m_vec_rep = np.full(m_models, a_m_vec)
    Q_mat = np.outer(a_m_vec_rep, a_m_vec_rep); c_vec = v_m_vec + a_m_vec_rep**2
    obj_func = lambda w: w @ Q_mat @ w - c_vec @ w; jac_func = lambda w: 2 * Q_mat @ w - c_vec
    bounds_w = Bounds(np.zeros(m_models), np.full(m_models, np.inf))
    constraints_w = LinearConstraint(np.ones((1, m_models)), [1.0], [1.0])
    w_initial = np.ones(m_models) / m_models
    qp_res = minimize(obj_func, w_initial, method='SLSQP', jac=jac_func, bounds=bounds_w, constraints=constraints_w, options={'ftol': 1e-9, 'disp': False})
    if qp_res.success:
        w_opt_for_X = qp_res.x; w_opt_for_X = np.maximum(0, w_opt_for_X); w_opt_for_X /= np.sum(w_opt_for_X)
        var_a_m = 0.0; X_piY_val = w_opt_for_X @ v_m_vec + var_a_m
        return max(0, X_piY_val)
    else: return np.nan
def phi_and_grad(w, R_common, Sigma_alpha): # Added R_common argument
    V_list = [Sigma_alpha[:,:,m] - np.outer(R_common[:,0], R_common[:,0]) for m in range(M)]
    Vw = np.sum(np.fromiter((w[m] * V_list[m] for m in range(M)), dtype=object))
    try:
        Vw_reg = Vw + np.eye(K) * 1e-9
        y = la.solve(Vw_reg, R_common[:,0], assume_a='pos')
        phi = R_common[:,0] @ y
        grad = -np.array([y @ V_list[m] @ y for m in range(M)])
    except (np.linalg.LinAlgError, ValueError):
        try: Vw_pinv = np.linalg.pinv(Vw_reg); y = Vw_pinv @ R_common[:,0]; phi = R_common[:,0] @ y; grad = -np.array([y @ V_list[m] @ y for m in range(M)])
        except np.linalg.LinAlgError: return np.inf, np.full(M, np.nan)
    return phi, grad
def minimise_phi(R_common, Sigma_alpha, w0=None):
    w = proj_simplex(w0 if w0 is not None else np.ones(M) / M)
    phi, g = phi_and_grad(w, R_common, Sigma_alpha)
    if np.isnan(g).any(): return np.inf, w, 0
    lr = 0.1
    for it in range(1, MAX_OPT_ITERS + 1):
        proj_grad_norm = np.linalg.norm(w - proj_simplex(w - g))
        if proj_grad_norm < tol_grad: break
        w_prev = w.copy(); phi_prev = phi; alpha_ls = 0.3; beta_ls = 0.7; lr_curr = lr * 2
        accepted_step = False
        for _ls_iter in range(20):
             lr_curr *= beta_ls; w_new = proj_simplex(w - lr_curr * g)
             phi_new, g_new = phi_and_grad(w_new, R_common, Sigma_alpha)
             if np.isnan(g_new).any(): continue
             if phi_new <= phi_prev - 1e-9 * lr_curr * (g @ g):
                 w, phi, g = w_new, phi_new, g_new; lr = lr_curr; accepted_step = True; break
        if not accepted_step: break
    return phi, w, it
def calculate_bounds_and_sr_sigma_only(pi_star, w_star, H_star_val, R_common, Sigma_alpha, V_alpha):
    SR_rob_val, B_U1_val, B_U2_val = np.nan, np.nan, np.nan
    if pi_star is None or w_star is None or np.isnan(H_star_val): return SR_rob_val, B_U1_val, B_U2_val
    r_bar_ws = R_common[:,0]; numerator_sr = pi_star @ r_bar_ws
    denominator_sr_sq = H_star_val
    if denominator_sr_sq > 1e-12: SR_rob_val = numerator_sr / np.sqrt(denominator_sr_sq)
    elif abs(numerator_sr) < GENTOL: SR_rob_val = 0.0
    Vw_star = np.sum(np.fromiter((w_star[m] * V_alpha[:,:,m] for m in range(M)), dtype=object))
    try: Vw_star_inv = np.linalg.inv(Vw_star + np.eye(K)*1e-10); s_w_star_num = r_bar_ws @ Vw_star_inv @ r_bar_ws
    except np.linalg.LinAlgError: B_U1_val = np.nan; s_w_star_num = -1
    else:
        if s_w_star_num < -GENTOL: s_w_star_num = 0
        B_U1_val = np.sqrt(s_w_star_num)
    rho_m_vals = np.zeros(M); all_rho_m_ok = True
    for m in range(M):
        try: Sigma_m_inv = np.linalg.inv(Sigma_alpha[:,:,m] + np.eye(K)*1e-10); rho_m_vals[m] = r_bar_ws @ Sigma_m_inv @ r_bar_ws
        except np.linalg.LinAlgError: all_rho_m_ok = False; break
    if all_rho_m_ok:
        rho_max_val = np.max(rho_m_vals); rho_max_val = min(rho_max_val, 1.0 - 1e-12)
        if rho_max_val < 0: B_U2_val = np.nan
        else: B_U2_val = np.sqrt(rho_max_val / (1.0 - rho_max_val))
    else: B_U2_val = np.nan
    return SR_rob_val, B_U1_val, B_U2_val
def finite_diff_vec(v_p, v_m, h, length):
    if v_p is None or v_m is None or h is None or np.isnan(h) or abs(h) < FINITE_DIFF_EPS: return np.full(length, np.nan)
    actual_h = h if abs(h) > FINITE_DIFF_EPS else np.sign(h)*FINITE_DIFF_EPS if h!=0 else FINITE_DIFF_EPS
    return (v_p - v_m) / (2 * actual_h)

# --------------------------- α-sweep (sigma only case) ---------------------
def run_sweep_sigma_only_full_bounds(out_csv="sigma_only_sweep_full_bounds_dpi.csv"):
    log = logging.getLogger("sweep_sigma")
    log.propagate = False
    log.setLevel(logging.INFO)
    if not log.handlers: log.addHandler(logging.StreamHandler(sys.stdout))
    log.info("========== Sigma Only α-Sweep Full Bounds with dpi* START ==========")

    # *** ADD COLUMN DEFINITIONS HERE ***
    col_w   = [f"w*_m{m}" for m in range(M)]
    col_pi  = [f"pi*_k{k}" for k in range(K)]
    col_ret = [f"cstr_ret_m{m}" for m in range(M)]
    col_lam = [f"lam_m{m}" for m in range(M)]
    col_dpi = [f"dpi*_k{k}" for k in range(K)]
    columns = (["alpha", "H_star", "supp_w", "lambda_min_Vm", "iterations",
                "SR_rob", "SR_Bound_Sw", "SR_Bound_rho_max",
                "SR_Bound_Y0", "SR_Bound_XpiY",
                "Y0_val", "XpiY_val"]
               + col_w + col_pi + col_ret + col_lam + col_dpi)

    rows, cache = [], {}
    rng_y0 = np.random.default_rng(23456)
    t0 = datetime.now()
    y0_fail_count = 0; xpiy_fail_count = 0; phi_fail_count = 0

    def get_sigma_only_pi(alpha_val):
        cache_key = alpha_val
        if cache_key not in cache:
            try:
                R_a, Sigma_a, V_a, lambda_min_a = build_params_sigma_only(alpha_val)
                phi_star_a, w_star_a, iters_a = minimise_phi(R_a, Sigma_a)
                if np.isinf(phi_star_a): raise ValueError("minimise_phi failed")
                Vw_star_a = np.sum(np.fromiter((w_star_a[m] * V_a[:,:,m] for m in range(M)), dtype=object))
                Vw_star_a_inv = np.linalg.inv(Vw_star_a + np.eye(K)*1e-10)
                r_Vinv_r_a = R_a[:,0] @ Vw_star_a_inv @ R_a[:,0]
                if r_Vinv_r_a <= 1e-9: raise ValueError("r_Vinv_r near zero")
                pi_star_a = (mu_tilde * Vw_star_a_inv @ R_a[:,0]) / r_Vinv_r_a
                cache[cache_key] = {"pi_star": pi_star_a}
            except Exception as e: cache[cache_key] = None
        return cache[cache_key]

    for idx, alpha in enumerate(ALPHA_GRID):
        if (idx * 100 // N_ALPHA) > ((idx - 1) * 100 // N_ALPHA):
             log.info(f"Progress: {idx * 100 / N_ALPHA:.0f}% (α = {alpha:.5f})")
        try:
            R_th, Sigma_th, V_th, lambda_min_val = build_params_sigma_only(alpha)
            phi_star, w_star, nit = minimise_phi(R_th, Sigma_th)
            if np.isinf(phi_star): raise ValueError("minimise_phi failed")
            Vw_star = np.sum(np.fromiter((w_star[m] * V_th[:,:,m] for m in range(M)), dtype=object))
            Vw_star_inv = np.linalg.inv(Vw_star + np.eye(K)*1e-10)
            r_Vinv_r = R_th[:,0] @ Vw_star_inv @ R_th[:,0]
            if r_Vinv_r <= 1e-9: raise ValueError("r_Vinv_r near zero in main loop")
            pi_star = (mu_tilde * Vw_star_inv @ R_th[:,0]) / r_Vinv_r
            H_star = mu_tilde**2 / phi_star
        except Exception as e:
            log.warning(f"Skipping alpha = {alpha:.5f} due to error in main calc: {e}")
            phi_fail_count += 1
            continue
        if alpha not in cache: cache[alpha] = {"pi_star": pi_star}
        SR_rob, B_U1, B_U2 = calculate_bounds_and_sr_sigma_only(pi_star, w_star, H_star, R_th, Sigma_th, V_th)
        pi_Y, Y0_val = calculate_Y0(R_th, Sigma_th, mu_tilde, K, M, rng_y0)
        if pi_Y is None or np.isnan(Y0_val): y0_fail_count += 1
        XpiY_val = calculate_X_pi_Y(pi_Y, R_th, Sigma_th, V_th, K, M)
        if pi_Y is not None and np.isnan(XpiY_val): xpiy_fail_count += 1
        B_L1 = mu_tilde / np.sqrt(Y0_val) if Y0_val is not None and not np.isnan(Y0_val) and Y0_val > 1e-12 else np.nan
        B_L2 = mu_tilde / np.sqrt(XpiY_val) if XpiY_val is not None and not np.isnan(XpiY_val) and XpiY_val > 1e-12 else np.nan
        dpi_star = np.full(K, np.nan); prev_alpha = ALPHA_GRID[idx - 1] if idx > 0 else None
        next_alpha = ALPHA_GRID[idx + 1] if idx < N_ALPHA - 1 else None; h = np.nan; pi_p, pi_m = None, None
        if prev_alpha is not None and next_alpha is not None:
            h = min(alpha - prev_alpha, next_alpha - alpha); h = max(h, FINITE_DIFF_EPS)
            res_p = get_sigma_only_pi(alpha + h); res_m = get_sigma_only_pi(alpha - h)
            if res_p is not None: pi_p = res_p["pi_star"]
            if res_m is not None: pi_m = res_m["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / (2*h)
        elif prev_alpha is None and next_alpha is not None:
            h = next_alpha - alpha
            if h >= FINITE_DIFF_EPS: res_p = get_sigma_only_pi(next_alpha); pi_m = pi_star
            if res_p is not None: pi_p = res_p["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / h
        elif next_alpha is None and prev_alpha is not None:
            h = alpha - prev_alpha
            if h >= FINITE_DIFF_EPS: res_m = get_sigma_only_pi(prev_alpha); pi_p = pi_star
            if res_m is not None: pi_m = res_m["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / h
        cstr_ret = np.full(M, mu_tilde); lam_vec = np.full(M, np.nan)
        row_data = {"alpha": alpha, "H_star": H_star, "supp_w": int((w_star > 1e-6).sum()),
                    "lambda_min_Vm": lambda_min_val, "iterations": nit, "SR_rob": SR_rob,
                    "SR_Bound_Sw": B_U1, "SR_Bound_rho_max": B_U2, "SR_Bound_Y0": B_L1, "SR_Bound_XpiY": B_L2,
                    "Y0_val": Y0_val, "XpiY_val": XpiY_val}
        row_data.update({f"w*_m{m}": w_star[m] for m in range(M)})
        row_data.update({f"pi*_k{k}": pi_star[k] for k in range(K)})
        row_data.update({f"cstr_ret_m{m}": cstr_ret[m] for m in range(M)})
        row_data.update({f"lam_m{m}": lam_vec[m] for m in range(M)})
        row_data.update({f"dpi*_k{k}": dpi_star[k] for k in range(K)})
        rows.append(row_data)

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(out_csv, index=False, float_format="%.10g")
    log.info(f"CSV written to {out_csv}  (elapsed {(datetime.now() - t0).total_seconds():.1f}s)")
    log.info(f"Total Phi Minimization Fails: {phi_fail_count}, Y0 Fails: {y0_fail_count}, XpiY Fails: {xpiy_fail_count}")
    return df

# --------------------------- MAIN --------------------------------
if __name__ == "__main__":
    # Define ALPHA_MIN, ALPHA_MAX, N_ALPHA before calling
    # Example:
    ALPHA_MIN = 0.01
    ALPHA_MAX = 1.0
    N_ALPHA = 101 # Reduce N_ALPHA for testing
    ALPHA_GRID = np.linspace(ALPHA_MIN, ALPHA_MAX, N_ALPHA)

    df_preview = run_sweep_sigma_only_full_bounds()
    print(df_preview.head())

#$C^m$のみが異なる場合

In [ ]:
# -*- coding: utf-8 -*-
"""
corr_only_sweep_full_bounds_dpi.py
- Robust MVP (r, sigma fixed, C varies) α-sweep.
- Calculates bounds from Thm 2 (S(w*)), Thm 6 (Y0), Thm 8 (X_piY).
- Includes dpi* calculation.
Based on the provided correlation_only script.
"""
import numpy as np
import pandas as pd
from datetime import datetime
import scipy.linalg as la # Added for phi_and_grad check
from scipy.optimize import minimize, Bounds, LinearConstraint # For Y0, X_piY
import logging, sys # Added logging

# ---------------------------- GLOBALS ----------------------------
K, M = 4, 4
mu_tilde = 0.03

# ---------- optimiser hyper-parameters (Using corr_only values) ---
# Original phi_min had it_max=40, lbfgs_m=5. We'll use minimise_phi's defaults.
# Need tol_grad for stopping L-BFGS (use a reasonable default)
tol_grad = 1e-8
# --- Other constants ---
INNER_OPT_TOL_FEAS = 1e-9
SOLVER_FEAS_TOL    = 1e-7
GENTOL             = 1e-9
FINITE_DIFF_EPS    = 1e-12

# -------------- Fixed parameters --------------
r_bar = np.array([0.02, 0.03, 0.04, 0.05])
sigma_bar = np.array([0.20, 0.25, 0.30, 0.35])
inv_sigma_bar = 1.0 / sigma_bar
g_vec = r_bar / sigma_bar

# -------------- Correlation setup ----------------------
beta_corr = 1 # From original code

def block_corr(rho_in, rho_out): # Identical
    C = np.full((K, K), rho_out); np.fill_diagonal(C, 1.0)
    for i, j in ((0, 1), (2, 3)): C[i, j] = C[j, i] = rho_in
    return C
C_base = block_corr(0.75, 0.50)
C_tilde = [block_corr(0.85, 0.40), block_corr(0.65, 0.40),
           block_corr(0.65, 0.60), block_corr(0.85, 0.60)]

# ---- blend & SPD-project ----------------------------------------
def mix_corr(C0, C1, alpha, beta=0.5, eps=1e-4): # Identical
    C = (1 - beta * alpha) * C0 + beta * alpha * C1
    try:
        eigval, eigvec = np.linalg.eigh(C)
    except np.linalg.LinAlgError: # Handle potential convergence issues
        # print(f"Warning: Eigh failed for mix_corr alpha={alpha}. Using fallback.")
        return C0, -np.inf # Return base and signal error
    eigval = np.clip(eigval, eps, None)
    C_spd = eigvec @ np.diag(eigval) @ eigvec.T
    # Ensure diagonal is 1 for correlation matrix
    D_diag = np.sqrt(np.diag(C_spd))
    if np.any(D_diag <= 1e-10): # Avoid division by zero
         # print(f"Warning: mix_corr resulted in near-zero diagonal for alpha={alpha}")
         # Fallback? Return C0? Or regularized C_spd?
         # Let's return regularized C_spd but signal maybe lower quality min_eig
         C_spd += np.eye(K) * 1e-8
         D_diag = np.sqrt(np.diag(C_spd))
         min_eig_val = np.min(np.linalg.eigvalsh(C_spd / np.outer(D_diag, D_diag)))
         # Need to return correlation matrix
         C_corr = C_spd / np.outer(D_diag, D_diag)
         np.fill_diagonal(C_corr, 1.0)
         return C_corr, min_eig_val * 0.1 # Penalize lambda_min if fallback used

    D_inv = np.diag(1.0 / D_diag)
    C_corr = D_inv @ C_spd @ D_inv
    np.fill_diagonal(C_corr, 1.0) # Ensure diagonal is exactly 1
    return C_corr, float(np.min(eigval)) # Return original min eigenvalue before clipping for info

# ---------------------- simplex projection ------------------------
def proj_simplex(v): # Identical
    v = np.asarray(v) # Ensure numpy array
    v = v.clip(min=0.0) # Clip negatives first
    if abs(v.sum() - 1) < 1e-10: return v
    u = np.sort(v)[::-1]; cssv = np.cumsum(u) - 1
    try:
        # Handle edge case where all elements are zero after clipping
        if cssv[-1] < 0: return np.zeros_like(v) # Or equi-weight? Let's return zeros
        rho_candidates = np.where(u - cssv / (np.arange(len(v)) + 1) > 0)[0]
        if len(rho_candidates) == 0: # If check doesn't yield index (e.g., near zero input)
            # Heuristic: find last element > 0? Or assume rho=0?
            # Let's return equi-weight as fallback if rho cannot be found
            return np.ones_like(v) / len(v)
        rho = rho_candidates[-1]
        theta = cssv[rho] / (rho + 1)
        return np.maximum(v - theta, 0.0)
    except IndexError: return np.ones_like(v) / len(v) # Fallback

# ---------------------- parameter builder (corr varies) -----------
def build_params_corr_only(alpha):
    """Returns R(=r_bar repeated), Sigma_m(alpha), V_m(alpha)"""
    R_th = np.tile(r_bar[:, np.newaxis], (1, M))
    Sigma_th = np.zeros((K, K, M))
    V_th = np.zeros((K, K, M))
    lambda_min_corr_log = []
    D_sigma_bar = np.diag(sigma_bar)
    reg_V = 1e-8; reg_S = 1e-8

    for m in range(M):
        C_m, lam_min = mix_corr(C_base, C_tilde[m], alpha, beta=beta_corr, eps=1e-5)
        if lam_min == -np.inf : raise ValueError(f"mix_corr failed for alpha={alpha}, m={m}")
        lambda_min_corr_log.append(lam_min)

        V_m = D_sigma_bar @ C_m @ D_sigma_bar
        min_eig_Vm = np.min(np.linalg.eigvalsh(V_m))
        if min_eig_Vm <= reg_V: V_m += np.eye(K) * (abs(min_eig_Vm) + reg_V)
        V_th[:, :, m] = V_m

        Sigma_th[:, :, m] = V_m + np.outer(r_bar, r_bar)
        min_eig_Sm = np.min(np.linalg.eigvalsh(Sigma_th[:, :, m]))
        if min_eig_Sm <= reg_S: Sigma_th[:, :, m] += np.eye(K) * (abs(min_eig_Sm) + reg_S)

    lambda_min_for_csv = float(min(lambda_min_corr_log)) if lambda_min_corr_log else np.nan

    return R_th, Sigma_th, V_th, lambda_min_for_csv

# ---------------------- Utility Functions (Y0, X_piY) ---------------
# (Identical to previous correct versions, omitted for brevity)
def calculate_Y0(R_trial, Sigma_list_trial, mu_tilde_val, k_assets, m_models, rng_y0_pert):
    pi_avg_r = np.mean(R_trial, axis=1); k_assets=R_trial.shape[0]; m_models=R_trial.shape[1]
    if np.linalg.norm(pi_avg_r) > 1e-6: pi_initial_guess = pi_avg_r / np.linalg.norm(pi_avg_r)
    else: pi_initial_guess = np.ones(k_assets) / np.sqrt(k_assets)
    max_tries_feas = 10; found_feasible_pi = False; SOLVER_FEAS_TOL=1e-7
    constraint_func = lambda pi: pi @ R_trial[:, 0] - mu_tilde_val # Use first column for r_bar
    for i_try in range(max_tries_feas):
        if constraint_func(pi_initial_guess) >= -SOLVER_FEAS_TOL : found_feasible_pi = True; break
        else: current_ret = pi_initial_guess @ R_trial[:, 0]
        if current_ret > 1e-9: pi_initial_guess *= (mu_tilde_val / current_ret) * (1.01 + 0.1*i_try)
        else: pi_initial_guess += rng_y0_pert.normal(0, 0.1, size=k_assets); norm_pi = np.linalg.norm(pi_initial_guess); pi_initial_guess /= norm_pi if norm_pi > 1e-6 else 1
    if not found_feasible_pi: pi_initial_guess = np.ones(k_assets) / k_assets
    t_initial_guess = 0.0
    for m in range(m_models): t_initial_guess = max(t_initial_guess, pi_initial_guess @ Sigma_list_trial[:,:,m] @ pi_initial_guess)
    t_initial_guess = max(1e-6, t_initial_guess)
    x_initial = np.concatenate([pi_initial_guess, [t_initial_guess]])
    func = lambda x: x[-1]; cons = []
    for m in range(m_models): cons.append({'type': 'ineq', 'fun': lambda x, Sm=Sigma_list_trial[:,:,m]: x[-1] - x[:-1] @ Sm @ x[:-1]})
    cons.append({'type': 'ineq', 'fun': lambda x, rm=R_trial[:,0]: x[:-1] @ rm - mu_tilde_val})
    bounds = [(-np.inf, np.inf)] * k_assets + [(1e-8, np.inf)]
    opt_res_Y0 = minimize(func, x_initial, method='SLSQP', bounds=bounds, constraints=cons, options={'ftol': 1e-8, 'disp': False, 'maxiter': 1000})
    pi_Y = None; Y0_val = np.nan
    if opt_res_Y0.success:
        pi_sol, t_sol = opt_res_Y0.x[:-1], opt_res_Y0.x[-1]; feasible = True
        min_ret_check = pi_sol @ R_trial[:,0]
        max_quad_check = np.max([pi_sol @ Sigma_list_trial[:,:,m] @ pi_sol for m in range(m_models)])
        if min_ret_check < mu_tilde_val - SOLVER_FEAS_TOL: feasible = False
        if abs(t_sol - max_quad_check) > SOLVER_FEAS_TOL * (1 + abs(t_sol)) + 1e-7 : feasible = False
        if feasible: pi_Y = pi_sol; Y0_val = t_sol
    return pi_Y, Y0_val

def calculate_X_pi_Y(pi_Y, R_trial, Sigma_list_trial, V_list_trial, k_assets, m_models):
    if pi_Y is None: return np.nan
    v_m_vec = np.array([pi_Y @ V_list_trial[:,:,m] @ pi_Y for m in range(m_models)])
    a_m_vec = R_trial[:,0].T @ pi_Y; a_m_vec_rep = np.full(m_models, a_m_vec)
    Q_mat = np.outer(a_m_vec_rep, a_m_vec_rep); c_vec = v_m_vec + a_m_vec_rep**2
    obj_func = lambda w: w @ Q_mat @ w - c_vec @ w; jac_func = lambda w: 2 * Q_mat @ w - c_vec
    bounds_w = Bounds(np.zeros(m_models), np.full(m_models, np.inf))
    constraints_w = LinearConstraint(np.ones((1, m_models)), [1.0], [1.0])
    w_initial = np.ones(m_models) / m_models
    qp_res = minimize(obj_func, w_initial, method='SLSQP', jac=jac_func, bounds=bounds_w, constraints=constraints_w, options={'ftol': 1e-9, 'disp': False})
    if qp_res.success:
        w_opt_for_X = qp_res.x; w_opt_for_X = np.maximum(0, w_opt_for_X); w_opt_for_X /= np.sum(w_opt_for_X)
        var_a_m = 0.0; X_piY_val = w_opt_for_X @ v_m_vec + var_a_m
        return max(0, X_piY_val)
    else: return np.nan


# -------- Φ(w) = gᵀQ⁻¹g  and its gradient ∇Φ(w) ------------------
# (Function definition identical to original code)
def phi_and_grad(w, Cstack):
    # Ensure Cstack elements are well-defined
    if any(np.any(np.isnan(C)) or np.any(np.isinf(C)) for C in Cstack):
        # print("Warning: NaN/Inf in Cstack input to phi_and_grad")
        return np.inf, np.full(len(w), np.nan), None

    Q = sum(w[m] * Cstack[m] for m in range(M))
    # Check if Q is SPD before solving
    try:
        # Add regularization for stability
        Q_reg = Q + np.eye(K) * 1e-9
        # Use Cholesky factorization for solve - more stable for SPD
        L = la.cholesky(Q_reg, lower=True)
        y = la.solve_triangular(L.T, la.solve_triangular(L, g_vec, lower=True))
        # y = np.linalg.solve(Q_reg, g_vec) # Alternative if Cholesky fails
        phi = g_vec @ y
        grad = -np.array([y @ Cstack[m] @ y for m in range(M)])
    except (np.linalg.LinAlgError, ValueError): # Catch solve errors or non-SPD for Cholesky
         # Fallback with pseudo-inverse
         try:
             Q_pinv = np.linalg.pinv(Q_reg if 'Q_reg' in locals() else Q + np.eye(K) * 1e-9)
             y = Q_pinv @ g_vec
             phi = g_vec @ y
             grad = -np.array([y @ Cstack[m] @ y for m in range(M)])
         except np.linalg.LinAlgError: # If pseudo-inverse also fails
             return np.inf, np.full(len(w), np.nan), None # Indicate failure

    return phi, grad, y # Return y as well

# -------------- L-BFGS on the simplex (from original code) -------------------
# (Function definition identical to original code)
def minimise_phi(Cstack, it_max=40, lbfgs_m=5): # Increased it_max slightly
    w = np.full(M, 1 / M); s_hist, y_hist = [], []
    phi, grad, y = phi_and_grad(w, Cstack)
    if np.isnan(grad).any(): return np.inf, w, y, 0 # Initial fail

    for it in range(1, it_max + 1):
        if np.linalg.norm(proj_simplex(w - grad) - w) < tol_grad: break # Use global tol_grad
        q = grad.copy(); alpha_hist = []
        for s, y_h in reversed(list(zip(s_hist, y_hist))):
            rho = 1.0 / (y_h @ s + 1e-12) # Avoid division by zero
            a = rho * (s @ q); alpha_hist.append(a); q -= a * y_h
        if s_hist: gamma = (s_hist[-1] @ y_hist[-1]) / (y_hist[-1] @ y_hist[-1] + 1e-12); q *= gamma
        for s, y_h, a in zip(s_hist, y_hist, reversed(alpha_hist)):
            rho = 1.0 / (y_h @ s + 1e-12); beta = rho * (y_h @ q); q += s * (a - beta)
        d = -q; step = 1.0; accepted_step = False
        for _ls in range(20): # Line search
            w_new = proj_simplex(w + step * d)
            # Add try-except around phi_and_grad in line search
            try: phi_new, grad_new, y_new = phi_and_grad(w_new, Cstack)
            except: phi_new=np.inf; grad_new=None; y_new=None # Handle potential errors
            if np.isnan(phi_new) or (grad_new is not None and np.isnan(grad_new).any()): # Check for NaN
                 step *= 0.5; continue
            # Use slightly more robust Armijo check
            if phi_new < phi - 1e-5 * step * max(1e-9, grad @ d): # Adjust Armijo param if needed
                accepted_step = True; break
            step *= 0.5
        if not accepted_step: break
        s_k = w_new - w; y_k = grad_new - grad
        if y_k @ s_k > 1e-12:
            s_hist.append(s_k); y_hist.append(y_k)
            if len(s_hist) > lbfgs_m: s_hist.pop(0); y_hist.pop(0)
        w, phi, grad, y = w_new, phi_new, grad_new, y_new
    # Return minimized phi, w, y, and iterations
    return phi, w, y, it


# --------------------- CALCULATE BOUNDS AND SR (Corr only case) -----------------------
def calculate_bounds_and_sr_corr_only(pi_star, w_star, H_star_val, R_common, Sigma_alpha, V_alpha):
    """Calculate SR_rob, B_U1 (S(w*)), B_U2 (rho_max)."""
    SR_rob_val, B_U1_val, B_U2_val = np.nan, np.nan, np.nan
    if pi_star is None or w_star is None or np.isnan(H_star_val): return SR_rob_val, B_U1_val, B_U2_val

    # SR_rob: Use H_star_val = mu^2 / phi* as denominator^2
    r_bar_ws = R_common[:,0] # Since R is common
    numerator_sr = pi_star @ r_bar_ws # Should be mu_tilde by construction of pi_star
    denominator_sr_sq = H_star_val
    if denominator_sr_sq > 1e-12: SR_rob_val = numerator_sr / np.sqrt(denominator_sr_sq)
    elif abs(numerator_sr) < GENTOL: SR_rob_val = 0.0

    # B_U1 = sqrt(S(w*))
    # V^w = sum w_m V^m = D(sig_bar) (sum w_m C^m) D(sig_bar)
    C_w_star = np.sum(np.fromiter((w_star[m] * (la.inv(np.diag(sigma_bar)) @ V_alpha[:,:,m] @ la.inv(np.diag(sigma_bar))) for m in range(M)), dtype=object)) # Get weighted C
    Vw_star = np.diag(sigma_bar) @ C_w_star @ np.diag(sigma_bar)
    try:
        Vw_star_inv = np.linalg.inv(Vw_star + np.eye(K)*1e-10); s_w_star_num = r_bar_ws @ Vw_star_inv @ r_bar_ws
        if s_w_star_num < -GENTOL: s_w_star_num = 0; B_U1_val = np.sqrt(s_w_star_num)
    except np.linalg.LinAlgError: B_U1_val = np.nan

    # B_U2 = sqrt(rho_max / (1-rho_max))
    rho_m_vals = np.zeros(M); all_rho_m_ok = True
    for m in range(M):
        try: Sigma_m_inv = np.linalg.inv(Sigma_alpha[:,:,m] + np.eye(K)*1e-10); rho_m_vals[m] = r_bar_ws @ Sigma_m_inv @ r_bar_ws
        except np.linalg.LinAlgError: all_rho_m_ok = False; break
    if all_rho_m_ok:
        rho_max_val = np.max(rho_m_vals); rho_max_val = min(rho_max_val, 1.0 - 1e-12)
        if rho_max_val < 0: B_U2_val = np.nan
        else: B_U2_val = np.sqrt(rho_max_val / (1.0 - rho_max_val))
    else: B_U2_val = np.nan
    return SR_rob_val, B_U1_val, B_U2_val

# ------------------------ FINITE DIFFERENCE HELPER -----------------------
def finite_diff_vec(v_p, v_m, h, length):
    if v_p is None or v_m is None or h is None or np.isnan(h) or abs(h) < FINITE_DIFF_EPS: return np.full(length, np.nan)
    actual_h = h if abs(h) > FINITE_DIFF_EPS else np.sign(h)*FINITE_DIFF_EPS if h!=0 else FINITE_DIFF_EPS
    return (v_p - v_m) / (2 * actual_h)

# ------------------------ α-sweep main computation -----------------------
def run_sweep_corr_only_full_bounds(out_csv="corr_only_sweep_full_bounds_dpi.csv"):
    log = logging.getLogger("sweep_corr")
    log.propagate = False
    log.setLevel(logging.INFO)
    if not log.handlers: log.addHandler(logging.StreamHandler(sys.stdout))
    log.info("========== Correlation Only α-Sweep Full Bounds with dpi* START ==========")

    col_w   = [f"w*_m{m}" for m in range(M)]; col_pi  = [f"pi*_k{k}" for k in range(K)]
    col_ret = [f"cstr_ret_m{m}" for m in range(M)]; col_lam = [f"lam_m{m}" for m in range(M)]
    col_dpi = [f"dpi*_k{k}" for k in range(K)]
    columns = (["alpha", "H_star", "supp_w", "lambda_min_corr", "iterations", # lambda_min from mix_corr
                "SR_rob", "SR_Bound_Sw", "SR_Bound_rho_max",
                "SR_Bound_Y0", "SR_Bound_XpiY",
                "Y0_val", "XpiY_val"]
               + col_w + col_pi + col_ret + col_lam + col_dpi)

    rows, cache = [], {}
    rng_y0 = np.random.default_rng(34567)
    t0 = datetime.now()
    y0_fail_count = 0; xpiy_fail_count = 0; phi_fail_count = 0

    # --- Define get helper for caching pi_star ---
    def get_corr_only_pi(alpha_val):
        cache_key = alpha_val
        if cache_key not in cache:
            try:
                R_a, Sigma_a, V_a, lambda_min_a = build_params_corr_only(alpha_val)
                Cstack_a = [(la.inv(np.diag(sigma_bar)) @ V_a[:,:,m] @ la.inv(np.diag(sigma_bar))) for m in range(M)] # Extract C_m from V_m
                phi_star_a, w_star_a, y_star_a, iters_a = minimise_phi(Cstack_a)
                if np.isinf(phi_star_a): raise ValueError("minimise_phi failed")
                pi_star_a = (mu_tilde / phi_star_a) * inv_sigma_bar * y_star_a
                cache[cache_key] = {"pi_star": pi_star_a}
            except Exception as e: cache[cache_key] = None
        return cache[cache_key]
    # --- End get helper ---

    for idx, alpha in enumerate(ALPHA_GRID):
        if (idx * 100 // N_ALPHA) > ((idx - 1) * 100 // N_ALPHA):
             log.info(f"Progress: {idx * 100 / N_ALPHA:.0f}% (α = {alpha:.5f})")

        try:
            R_th, Sigma_th, V_th, lambda_min_val = build_params_corr_only(alpha)
            # Calculate Cstack for minimise_phi
            Cstack = []
            for m in range(M):
                # Extract C_m from V_m = D C_m D
                D_inv = np.diag(inv_sigma_bar)
                C_m = D_inv @ V_th[:,:,m] @ D_inv
                # Optional: Re-normalize C_m to ensure perfect correlation matrix properties?
                # diag_C = np.sqrt(np.diag(C_m))
                # if np.any(diag_C <= 1e-10): raise ValueError("Zero diag in extracted C_m")
                # C_m = np.diag(1.0/diag_C) @ C_m @ np.diag(1.0/diag_C)
                # np.fill_diagonal(C_m, 1.0)
                Cstack.append(C_m)

            phi_star, w_star, y_star, nit = minimise_phi(Cstack)
            if np.isinf(phi_star): raise ValueError("minimise_phi failed")

            H_star = mu_tilde ** 2 / phi_star
            pi_star = (mu_tilde / phi_star) * inv_sigma_bar * y_star

        except Exception as e:
            log.warning(f"Skipping alpha = {alpha:.5f} due to error in main calc: {e}")
            phi_fail_count += 1
            continue

        if alpha not in cache: cache[alpha] = {"pi_star": pi_star}

        SR_rob, B_U1, B_U2 = calculate_bounds_and_sr_corr_only(pi_star, w_star, H_star, R_th, Sigma_th, V_th)
        pi_Y, Y0_val = calculate_Y0(R_th, Sigma_th, mu_tilde, K, M, rng_y0)
        if pi_Y is None or np.isnan(Y0_val): y0_fail_count += 1
        XpiY_val = calculate_X_pi_Y(pi_Y, R_th, Sigma_th, V_th, K, M)
        if pi_Y is not None and np.isnan(XpiY_val): xpiy_fail_count += 1
        B_L1 = mu_tilde / np.sqrt(Y0_val) if Y0_val is not None and not np.isnan(Y0_val) and Y0_val > 1e-12 else np.nan
        B_L2 = mu_tilde / np.sqrt(XpiY_val) if XpiY_val is not None and not np.isnan(XpiY_val) and XpiY_val > 1e-12 else np.nan

        # Calculate dpi* using finite difference and cache
        dpi_star = np.full(K, np.nan); prev_alpha = ALPHA_GRID[idx - 1] if idx > 0 else None
        next_alpha = ALPHA_GRID[idx + 1] if idx < N_ALPHA - 1 else None; h = np.nan; pi_p, pi_m = None, None
        if prev_alpha is not None and next_alpha is not None:
            h = min(alpha - prev_alpha, next_alpha - alpha); h = max(h, FINITE_DIFF_EPS)
            res_p = get_corr_only_pi(alpha + h); res_m = get_corr_only_pi(alpha - h)
            if res_p is not None: pi_p = res_p["pi_star"]
            if res_m is not None: pi_m = res_m["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / (2*h)
        elif prev_alpha is None and next_alpha is not None:
            h = next_alpha - alpha
            if h >= FINITE_DIFF_EPS: res_p = get_corr_only_pi(next_alpha); pi_m = pi_star
            if res_p is not None: pi_p = res_p["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / h
        elif next_alpha is None and prev_alpha is not None:
            h = alpha - prev_alpha
            if h >= FINITE_DIFF_EPS: res_m = get_corr_only_pi(prev_alpha); pi_p = pi_star
            if res_m is not None: pi_m = res_m["pi_star"]
            if pi_p is not None and pi_m is not None: dpi_star = (pi_p - pi_m) / h

        # Format other outputs
        cstr_ret = np.full(M, mu_tilde); lam_vec = np.full(M, np.nan)

        # Append row
        row_data = {"alpha": alpha, "H_star": H_star, "supp_w": int((w_star > 1e-6).sum()),
                    "lambda_min_corr": lambda_min_val, "iterations": nit, "SR_rob": SR_rob,
                    "SR_Bound_Sw": B_U1, "SR_Bound_rho_max": B_U2, "SR_Bound_Y0": B_L1, "SR_Bound_XpiY": B_L2,
                    "Y0_val": Y0_val, "XpiY_val": XpiY_val}
        row_data.update({f"w*_m{m}": w_star[m] for m in range(M)})
        row_data.update({f"pi*_k{k}": pi_star[k] for k in range(K)})
        row_data.update({f"cstr_ret_m{m}": cstr_ret[m] for m in range(M)})
        row_data.update({f"lam_m{m}": lam_vec[m] for m in range(M)})
        row_data.update({f"dpi*_k{k}": dpi_star[k] for k in range(K)})
        rows.append(row_data)

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(out_csv, index=False, float_format="%.10g")
    log.info(f"CSV written to {out_csv}  (elapsed {(datetime.now() - t0).total_seconds():.1f}s)")
    log.info(f"Total Phi Minimization Fails: {phi_fail_count}, Y0 Fails: {y0_fail_count}, XpiY Fails: {xpiy_fail_count}")
    return df

# --------------------------- MAIN --------------------------------
if __name__ == "__main__":
    # Define ALPHA_MIN, ALPHA_MAX, N_ALPHA before calling
    ALPHA_MIN = 0.01
    ALPHA_MAX = 1.0
    N_ALPHA = 101 # Reduce N_ALPHA for testing
    ALPHA_GRID = np.linspace(ALPHA_MIN, ALPHA_MAX, N_ALPHA)

    df_preview = run_sweep_corr_only_full_bounds()
    print(df_preview.head())